# #크롤링 #구글맵스 #서울특별시 #종로구 #빵

In [1]:
import pandas as pd # 판다스
import time # 시간차로 인한 에러방지
import random # 시간차로 인한 에러방지
from tqdm import tqdm # 코드가 멋져짐
from selenium.webdriver.common.by import By # 경로추적
from selenium import webdriver # 셀레니움
from bs4 import BeautifulSoup # 뷰숲
import requests # 뷰숲
from numpy import NaN, NAN, nan # 결측치

#### 22. [최종] 마지막 스크롤까지 확실히 클릭.

In [2]:
import pandas as pd # 판다스
import time # 시간차로 인한 에러방지
import random
from tqdm import tqdm # 코드가 멋져짐
from selenium.webdriver.common.by import By # 경로추적
from selenium.webdriver.common.keys import Keys # 엔터키
from selenium import webdriver # 셀레니움
from numpy import NaN, NAN, nan # 결측치
from selenium.webdriver.edge.service import Service # 접근제한 우회
from selenium.webdriver.edge.options import Options # 접근제한 우회
from selenium.webdriver.support.ui import WebDriverWait # 기다리기
from selenium.webdriver.support import expected_conditions as EC # 요소 뜰 때까지 대기
from selenium.common.exceptions import TimeoutException # 시간 예외처리
from datetime import datetime

# 기본 정보 찾기 함수
def find_text(col, url):
    try:
        elements = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, url)))
        for element in elements:
            element = element.text
            col.append(element)  # 수정: 이 부분을 for 루프 안으로 이동
    except:
        col.append('정보없음')  # 수정: 예외 상황에서 '정보없음'을 리스트에 추가

# 빈데이터프레임, 클래스
info_df = pd.DataFrame()
review_df = pd.DataFrame()
merged_df = pd.DataFrame()

# Edge WebDriver의 경로를 지정합니다. '경로/edgedriver'를 실제 경로로 교체해주세요.
service = Service(executable_path='C:\\selenium\\msedgedriver.exe')
options = webdriver.EdgeOptions()
options.use_chromium = True  # Chromium 기반 Edge 사용을 명시합니다.
# 사용자 에이전트 문자열을 설정합니다.
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36"
options.add_argument(f'user-agent={user_agent}')

# Edge 드라이버를 초기화합니다.
driver = webdriver.Edge(service=service, options=options)

# 구글 맵 들어가기
url = f'https://www.google.co.kr/maps/?hl=ko&entry=ttu'
driver.get(url)

time.sleep(random.uniform(2,3))
search_box = driver.find_element(By.XPATH, '//*[@id="searchboxinput"]')
search_box.send_keys("서울특별시 은평구 빵")
search_box.send_keys(Keys.ENTER)

i = 3  # [step1] 인포, 리뷰, "클릭" 순으로 변경한다.
number = 1
while True:
    time.sleep(random.uniform(3,4))  # 구글 들어가서 대기
    click_elements = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.XPATH, f'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{i}]/div/a')))
    for element in click_elements:
        element.click()
    # driver.find_element(By.XPATH, f'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{i}]/div/a').click()

    # 주의하자. 반복되는 건 전역변수/지역변수의 문제다.
    name = []
    rating = []
    review = []
    category = []
    adress = []

    time.sleep(random.uniform(2,3))
    find_text(name, ".DUwDvf.lfPIob") # 이름에서 막히면 restart하세요.
    find_text(rating, ".F7nice > span:nth-child(1)")
    find_text(review, ".F7nice > span:nth-child(2)")
    find_text(category, ".fontBodyMedium span button.DkEaL")
    find_text(adress, ".rogA2c")

    # 임시 데이터프레임 (concat된 후 리셋됨)
    time.sleep(random.uniform(2,3))
    tmp = pd.DataFrame()
    tmp['number'] = [number]
    tmp['Store'] = name
    tmp['Review_score'] = rating
    tmp['Review_counts'] = review
    tmp['Category'] = category
    tmp['Address'] = adress[0]

    # 최종 데이터프레임은 전역변수에 들어감 > 빈 DF에 덮어쓰기
    time.sleep(random.uniform(2,3))
    info_df = pd.concat([info_df, tmp])
    
    # [step2] 리뷰 버튼 클릭 전 대기 [청수당, 아티장 에러주의]
    # time.sleep(random.uniform(3,5))
    # driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[3]/div/div/button[2]').click()

    try:
        # WebDriverWait과 expected_conditions를 사용하여 요소가 나타날 때까지 최대 10초간 대기
        review_elements = WebDriverWait(driver, 100).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[3]/div/div/button[2]')))
        for element in review_elements:
            element.click()  # 요소가 발견되면 클릭
    # except TimeoutException:
    #     print("해당 요소를 찾는 데 시간이 너무 오래 걸립니다.")
    except:
        pass # 리뷰 버튼도 정보 버튼도 없는 상태

    # element = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[3]/div/div/button[2]')
    # driver.execute_script("arguments[0].click();", element)

    # 정렬 버튼 클릭하기 전 대기
    try:
        result = []  # 새로운 리뷰만 넣을 건데, 중복을 제거하는 조건을 넣어야 한다.
        time_list = []

        time.sleep(random.uniform(3,4))
        driver.find_element(By.CSS_SELECTOR, 'div.m6QErb.Pf6ghf.KoSBEe.ecceSd.tLjsW > div.TrU0dc.kdfrQc').click()

        # 최신순 클릭하기 전 대기
        time.sleep(random.uniform(1,3))
        driver.find_element(By.XPATH, '//*[@id="action-menu"]/div[2]').click()

        # result = []  # 새로운 리뷰만 넣을 건데, 중복을 제거하는 조건을 넣어야 한다.
        # time_list = []

        time.sleep(random.uniform(2,4))  # 클릭하고 대기시간 필요하다. [에러주의]
        scroll = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[3]')
        last_height = driver.execute_script("return arguments[0].scrollHeight", scroll)
        num = 0
        while True:
            driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", scroll)
            time.sleep(random.uniform(4, 5))  # 새 리뷰가 로드되기를 기다린다.
            new_height = driver.execute_script("return arguments[0].scrollHeight", scroll)
            buttons = driver.find_elements(By.CSS_SELECTOR, ".w8nwRe.kyuRq")
            for button in buttons:
                if button.is_displayed():
                    if button.is_enabled():
                        # 버튼이 활성화되어 있으면 클릭
                        button.click()
                        num+=1
                        time.sleep(random.uniform(1, 2))
                    break  # 첫 번째 활성화된 버튼을 클릭한 후 반복을 멈춤
            # new_height = driver.execute_script("return arguments[0].scrollHeight", scroll) 이동
            if new_height == last_height:
                break  # 못누르고 무한 루프, 자세히 누르면 높이가 계속 같지 않음.
            last_height = new_height

        buttons = driver.find_elements(By.CSS_SELECTOR, ".w8nwRe.kyuRq")
        for button in buttons:
            if button.is_displayed():
                if button.is_enabled():
                    button.click()  # 마지막 버튼 있으면 클릭
                    num+=1
                    time.sleep(random.uniform(1, 2))
        print(f"자세히 버튼을 {num}번 클릭했습니다.")

        last_review = None  # 비교하는 변수를 추가한다. WOW
        flag = False  # 종료 플래그를 사용해본다.
        while not flag:
            # time.sleep(random.uniform(1, 2))  # 페이지 로드를 여유두고 기다린다.
            # element = driver.find_element(By.CSS_SELECTOR, '#QA0Szd')  # 소름 돋는 점. 필요 없는 코드다.
            reviews = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.wiI7pd')))
            # reviews = driver.find_elements(By.CSS_SELECTOR, '.wiI7pd')  # 현재 로드된 모든 리뷰를 찾는다. elements로 가져와서 len=1의 리스트가 됐다.
            
            if not reviews:  # 강북상회(1) 리뷰가 없다면 루프를 종료한다.
                break

            for review in reviews:
                review_text = review.text
                if review_text == last_review:  # 마지막으로 확인한 리뷰와 같다면 새 리뷰 로드를 기다린다.
                    break
                if review_text not in result:
                    if review.tag_name.lower() == 'span':
                        result.append(review_text)
                        now = datetime.now().strftime('%Y-%m-%d - %H:%M:%S')
                        time_list.append(now)
                else:
                    flag=True
            
            if len(result) == 1:  # 로드된 리뷰가 하나만 있다면, 더 이상 새 리뷰가 없는 것으로 간주하고 루프를 종료한다.
                break
                
            last_review = result[-1] if result else None  # 마지막 리뷰를 확인용 변수에 업데이트 한다. result가 빈값으면(False) None을 쓴다.
        
        # 강북빵상회(2)
        if result:
            # 리뷰 데이터프레임을 만든다.
            tmp2 = pd.DataFrame()
            tmp2['number'] = [number]*len(result)
            tmp2['Store'] = name*len(result)
            tmp2['Review_text'] = result
            tmp2['Time'] = time_list

            # 리뷰 데이터프레임을 적재한다.
            time.sleep(random.uniform(1,2))
            review_df = pd.concat([review_df, tmp2])
    except:
        pass  # 정렬 버튼이 없으면 지나가시오.
    
    print(f"{number}번째 {name}상점에서 {len(result)}개의 리뷰를 수집했습니다.")  # 파머스반브래드 12개> 28개

    # [step3] 다음 가게 번호 부여하고 스크롤을 이동한다.
    time.sleep(random.uniform(1,2))
    try:
        i+=2
        number+=1
        next_element = driver.find_element(By.XPATH, f'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{i}]/div/a')
        driver.execute_script("arguments[0].scrollIntoView(true);", next_element) # element 위치로 이동 > 수집 반복
    except:
        break

# 통합 데이터프레임인데 info 정보없음, 중복 오류나서 보류
merged_df = pd.merge(info_df, review_df, on=["number"], how="outer")

자세히 버튼을 2번 클릭했습니다.
1번째 ['도돌이표']상점에서 11개의 리뷰를 수집했습니다.
2번째 ['뚜레쥬르 녹번역점']상점에서 0개의 리뷰를 수집했습니다.
자세히 버튼을 4번 클릭했습니다.
3번째 ['뚜레쥬르 카페응암점']상점에서 8개의 리뷰를 수집했습니다.
자세히 버튼을 1번 클릭했습니다.
4번째 ['파리바게뜨']상점에서 29개의 리뷰를 수집했습니다.
자세히 버튼을 1번 클릭했습니다.
5번째 ['시월구일 베이커리']상점에서 11개의 리뷰를 수집했습니다.
자세히 버튼을 0번 클릭했습니다.
6번째 ['뚜레쥬르 역촌2동점']상점에서 8개의 리뷰를 수집했습니다.
자세히 버튼을 1번 클릭했습니다.
7번째 ['데일리베이크하우스']상점에서 3개의 리뷰를 수집했습니다.
자세히 버튼을 1번 클릭했습니다.
8번째 ['뚜레쥬르 은평구청점']상점에서 4개의 리뷰를 수집했습니다.
자세히 버튼을 0번 클릭했습니다.
9번째 ['박성원베이커리']상점에서 5개의 리뷰를 수집했습니다.
자세히 버튼을 0번 클릭했습니다.
10번째 ['오늘의 빵']상점에서 1개의 리뷰를 수집했습니다.
11번째 ['빵드밀역촌점']상점에서 0개의 리뷰를 수집했습니다.
자세히 버튼을 2번 클릭했습니다.
12번째 ['파리바게뜨']상점에서 29개의 리뷰를 수집했습니다.
13번째 ['하얀풍차']상점에서 0개의 리뷰를 수집했습니다.
14번째 ['빵이야기']상점에서 0개의 리뷰를 수집했습니다.
자세히 버튼을 2번 클릭했습니다.
15번째 ['뚜레쥬르갈현시장점']상점에서 13개의 리뷰를 수집했습니다.
자세히 버튼을 1번 클릭했습니다.
16번째 ['파리바게뜨역말사거리점']상점에서 25개의 리뷰를 수집했습니다.
자세히 버튼을 1번 클릭했습니다.
17번째 ['파리바게트 증산역점']상점에서 22개의 리뷰를 수집했습니다.
자세히 버튼을 3번 클릭했습니다.
18번째 ['연신내빵집']상점에서 3개의 리뷰를 수집했습니다.
19번째 ['송베이커리']상점에서 0개의 리뷰를 수집했습니다.
자세히 버튼을 0번 클릭했습니다.
20번째 ['빵재미

In [3]:
info_df  # 코드 중에 실행시키면 크롤링 씹힌다.

,number,Store,Review_score,Review_counts,Category,Address
0,1,도돌이표,4.7,(14),제과점,서울특별시 은평구 진관3로 70 823동 상가 106호
0,2,뚜레쥬르 녹번역점,정보없음,정보없음,제과점,서울특별시 은평구 응암동 1-12
0,3,뚜레쥬르 카페응암점,4.6,(11),제과점,서울특별시 은평구 응암로 277
0,4,파리바게뜨,3.9,(50),제과점,서울특별시 은평구 녹번동 117-111
0,5,시월구일 베이커리,4.2,(15),제과점,서울특별시 은평구 응암동 은평로16길 22
...,...,...,...,...,...,...
0,116,럭키문,5.0,(3),카페,서울특별시 은평구 은평로12길 8-4
0,117,메가커피 구산역점,4.3,(64),커피숍/커피 전문점,서울특별시 은평구 구산동 연서로 137
0,118,스타벅스 은평구청입구점,4.1,(810),커피숍/커피 전문점,서울특별시 은평구 은평로 170 상현빌딩 1층
0,119,할리스 연신내점,4.2,(464),카페,서울특별시 은평구 연서로 212


In [4]:
review_df

,number,Store,Review_text,Time
0,1,도돌이표,맛있어요.,2024-04-07 - 08:25:08
1,1,도돌이표,은평뉴타운 최고의 빵집. 성수동빵집/이촌동빵집 저리가라입니다. 일단 다 맛있어요,2024-04-07 - 08:25:08
2,1,도돌이표,제가 좋아하는 빵집이에요~^^오늘은 가르치는 학생들에게 크리스마스 선물로 미리...,2024-04-07 - 08:25:08
3,1,도돌이표,가게 크기랑 내부가 귀엽다. 빵 종류는 유럽+한국 퓨전 스타일. 다양하고 괜찮다. ...,2024-04-07 - 08:25:08
4,1,도돌이표,"가격은 주변에 비해 다소 높지만, 매일 신선한 건강하고 맛있는 빵을 먹을 수 있습니다.",2024-04-07 - 08:25:08
...,...,...,...,...
3,120,커핀그루나루 응암점,빵 커피 소금빵 맛있어요,2024-04-07 - 10:30:31
4,120,커핀그루나루 응암점,루프탑도 있고 커피도 맛있어요. 에이드랑 쥬스도 알알이 과육도 있고 맛있었구요..\...,2024-04-07 - 10:30:31
5,120,커핀그루나루 응암점,뷰가 좋은,2024-04-07 - 10:30:31
6,120,커핀그루나루 응암점,조타,2024-04-07 - 10:30:31


In [5]:
# 1. 정보없음 가게체크
info_df[info_df['Store'].str.contains("정보")]

,number,Store,Review_score,Review_counts,Category,Address


In [6]:
# 2. 인포, 리뷰 중복체크
info_df['Store'].value_counts()

Store
파리바게뜨 은평행복          2
파리바게뜨 역촌2호          2
씨엘                  2
빵이야기                2
바로굼빵터               2
                   ..
파리바게뜨 신사오거리점        1
팡야빵야                1
빵쭈루                 1
프라빈 독바위점 Prabean    1
커핀그루나루 응암점          1
Name: count, Length: 111, dtype: int64

In [7]:
# 2-1. 중복체크 데이터프레임
info_df[info_df['Store'].map(info_df['Store'].value_counts()) >= 2]

,number,Store,Review_score,Review_counts,Category,Address
0,4,파리바게뜨,3.9,(50),제과점,서울특별시 은평구 녹번동 117-111
0,12,파리바게뜨,4.2,(65),제과점,서울특별시 은평구 갈현동 서오릉로 200
0,14,빵이야기,정보없음,정보없음,제과점,좌측 750 12번지 KR 서울특별시 은평구 응암동 1층
0,20,빵재미,4.9,(7),제과점,서울특별시 은평구 녹번동 118-50번지 1층
0,21,빵재미,4.9,(7),제과점,서울특별시 은평구 녹번동 118-50번지 1층
0,29,바로굼빵터,1.0,(1),제과점,서울특별시 은평구 불광동 310-136번지 1층
0,54,빵이야기,4.0,(1),제과점,서울특별시 은평구 대조동 통일로77길 17
0,57,델리명과,5.0,(1),제과점,응암동 589-37번지 지상1층 은평구 서울특별시 KR
0,58,파리바게뜨 역촌2호,3.5,(17),제과점,서울특별시 은평구 역촌동 79-32
0,59,파리바게뜨 은평행복,정보없음,정보없음,제과점,서울특별시 은평구 응암동 124-1


In [8]:
# 2-2. 중복체크 데이터프레임
info_df.groupby('Store').filter(lambda x: len(x) >= 2)

,number,Store,Review_score,Review_counts,Category,Address
0,4,파리바게뜨,3.9,(50),제과점,서울특별시 은평구 녹번동 117-111
0,12,파리바게뜨,4.2,(65),제과점,서울특별시 은평구 갈현동 서오릉로 200
0,14,빵이야기,정보없음,정보없음,제과점,좌측 750 12번지 KR 서울특별시 은평구 응암동 1층
0,20,빵재미,4.9,(7),제과점,서울특별시 은평구 녹번동 118-50번지 1층
0,21,빵재미,4.9,(7),제과점,서울특별시 은평구 녹번동 118-50번지 1층
0,29,바로굼빵터,1.0,(1),제과점,서울특별시 은평구 불광동 310-136번지 1층
0,54,빵이야기,4.0,(1),제과점,서울특별시 은평구 대조동 통일로77길 17
0,57,델리명과,5.0,(1),제과점,응암동 589-37번지 지상1층 은평구 서울특별시 KR
0,58,파리바게뜨 역촌2호,3.5,(17),제과점,서울특별시 은평구 역촌동 79-32
0,59,파리바게뜨 은평행복,정보없음,정보없음,제과점,서울특별시 은평구 응암동 124-1


In [9]:
# 2-3. 중복체크 시리즈에 불리언식을 적용했다.
value_counts = info_df['Store'].value_counts()
value_counts[value_counts >= 2]

Store
파리바게뜨 은평행복    2
파리바게뜨 역촌2호    2
씨엘            2
빵이야기          2
바로굼빵터         2
델리명과          2
빵재미           2
챗베이커          2
파리바게뜨         2
Name: count, dtype: int64

In [10]:
# 2-4. 중복체크 시리즈에 람다함수를 적용했다.
info_df['Store'].value_counts()[lambda x: x >= 2]

Store
파리바게뜨 은평행복    2
파리바게뜨 역촌2호    2
씨엘            2
빵이야기          2
바로굼빵터         2
델리명과          2
빵재미           2
챗베이커          2
파리바게뜨         2
Name: count, dtype: int64

In [15]:
# 3. merge row counts =review_df row counts 개수체크 (inner, outer)
merged_df = pd.merge(info_df, review_df, on=["number"], how="outer")
merged_df

,number,Store_x,Review_score,Review_counts,Category,Address,Store_y,Review_text,Time
0,1,도돌이표,4.7,(14),제과점,서울특별시 은평구 진관3로 70 823동 상가 106호,도돌이표,맛있어요.,2024-04-07 - 08:25:08
1,1,도돌이표,4.7,(14),제과점,서울특별시 은평구 진관3로 70 823동 상가 106호,도돌이표,은평뉴타운 최고의 빵집. 성수동빵집/이촌동빵집 저리가라입니다. 일단 다 맛있어요,2024-04-07 - 08:25:08
2,1,도돌이표,4.7,(14),제과점,서울특별시 은평구 진관3로 70 823동 상가 106호,도돌이표,제가 좋아하는 빵집이에요~^^오늘은 가르치는 학생들에게 크리스마스 선물로 미리...,2024-04-07 - 08:25:08
3,1,도돌이표,4.7,(14),제과점,서울특별시 은평구 진관3로 70 823동 상가 106호,도돌이표,가게 크기랑 내부가 귀엽다. 빵 종류는 유럽+한국 퓨전 스타일. 다양하고 괜찮다. ...,2024-04-07 - 08:25:08
4,1,도돌이표,4.7,(14),제과점,서울특별시 은평구 진관3로 70 823동 상가 106호,도돌이표,"가격은 주변에 비해 다소 높지만, 매일 신선한 건강하고 맛있는 빵을 먹을 수 있습니다.",2024-04-07 - 08:25:08
...,...,...,...,...,...,...,...,...,...
1914,120,커핀그루나루 응암점,4.6,(10),커피숍/커피 전문점,서울특별시 은평구 응암로22길 29-1 2층,커핀그루나루 응암점,빵 커피 소금빵 맛있어요,2024-04-07 - 10:30:31
1915,120,커핀그루나루 응암점,4.6,(10),커피숍/커피 전문점,서울특별시 은평구 응암로22길 29-1 2층,커핀그루나루 응암점,루프탑도 있고 커피도 맛있어요. 에이드랑 쥬스도 알알이 과육도 있고 맛있었구요..\...,2024-04-07 - 10:30:31
1916,120,커핀그루나루 응암점,4.6,(10),커피숍/커피 전문점,서울특별시 은평구 응암로22길 29-1 2층,커핀그루나루 응암점,뷰가 좋은,2024-04-07 - 10:30:31
1917,120,커핀그루나루 응암점,4.6,(10),커피숍/커피 전문점,서울특별시 은평구 응암로22길 29-1 2층,커핀그루나루 응암점,조타,2024-04-07 - 10:30:31


In [16]:
# 4. 저장코드
info_df.to_csv('google_Enpyeong_in.csv', index=False)
review_df.to_csv('google_Enpyeong_re.csv', index=False)
merged_df.to_csv('google_Enpyeong.csv', index=False)

In [ ]:
# 1. 데이터 편집하기
info_df.loc[57] = ['파리바게트 서교점', '3.9', '126', '제과점', '서울특별시 마포구 월드컵북로 34']
# 2. 행 삭제하기
info_df = info_df.drop(107, axis=0)
# 3. 인덱스 리셋하며 새로 만들지 말기
info_df = info_df.reset_index(drop=True)
# 4. 파일 읽기
info_df = pd.read_csv('./google_Dongdaemun_in.csv')
review_df = pd.read_csv('./google_Dongdaemun_re.csv')
# 5. 정보없음 대체값
info_df['Store'] = info_df['Store'].str.replace('정보없음', '파리바게뜨 서교점')
info_df['Address'] = info_df['Address'].str.replace('정보없음', '서울특별시 마포구 월드컵북로 34')

In [13]:
# 1. 조건별 특정 인덱스의 리뷰 전문
review_df[review_df['Store'].str.contains('몽핀')].loc[128, 'Review_text']

'우연히 석파랑을 지나다 발견한 빵카페 몽핀. 어마무시한 곳. 일단 라떼 싱글 맛 합격 매우 합격. 다양한 빵종류 합격. 빵맛 베리매우 합격 대존맛. 가격대 천원대부터 다양한데 미친 갓성비.\n\n내부 넓이부터 압도되는 웅장함에 외부는 고즈넉한 한옥 석파랑과 별당이 있고, 몽핀 내부는 모던한 하나의 예술작품과도 같은 공간. 편안하면서도 고급진 분위기에 잔잔하며 빵빵한 스피커에 센스있는 선곡. 친절함 500% 무장하신 직원분들. 깔끔한 화장실. 이만한 카페는 정말 처음인 것 같습니다.\n\n지금 신상이라 아직 사람이 많지 않았는데, 이제 곧 미어터져 예약제로 운영하는 빵카페 될 포텐터지는 곳입니다. 그냥 어느 곳 하나 맘에 들지 않는 곳이 없습니다. 단점이 없습니다... 그냥 별 오백개 주고 싶은 곳이예요😃'

In [ ]:
# 2. 인덱스별 리뷰 전문
review_df['Review_text'][0]

In [16]:
# 3. 조건별 리뷰내용 전문보기 > CSV파일 있으면 해당 파일로 확인한다.
con1 = (review_df['Store'] == '파네트')
con2 = (review_df['Review_text'].str.contains('오다가다'))
print(review_df[con1&con2]['Review_text'][178])

오다가다 보이는 파네트에는 건물측면에 '자연발효'라는 써있는 큰 현수막이 걸려있었고 이것으로 빵을 파는 곳이라는 것을 알 수 있었다. 그러다 호기심에 늘어간 이 빵집에는 다른곳과 다른 점이 있었다. 많은 종류의 덩치가 끈 빵 옆에는 시식용 빵이 함께 놓여있다는 것이다. 프렌차이즈 빵에 질리거나 새롭고  질좋은 빵을 먹고 싶은 욕구가 있더라고 먹어본적이 없다면 사던것만 사기 마련이다. 그러나 시식을 하니 자연스레 구매하게 만든다..
실제 매번 먹어보고 그중 하나를 사서 나온다. 카페도 함께 있어 식사를 한다거나 디저트를 드실 수도 있다. 티라미스 쿠키 스콘 등도 판매한다. 넓은 공간 또한 답답함이 없다. 대표하는 빵도 꽤 블로그에 소개 되어 있어 참고할만 하다.
유일한 단점은 가격이 다소 비싸다는 것이다.


#### 21. '자세히'버튼 추가하기 > 자세히 누르면 나오는 서비스에 대한 정형화된 내용은 span에 해당되지 않더라 > 위코드로

In [1]:
import pandas as pd # 판다스
import time # 시간차로 인한 에러방지
import random
from tqdm import tqdm # 코드가 멋져짐
from selenium.webdriver.common.by import By # 경로추적
from selenium.webdriver.common.keys import Keys # 엔터키
from selenium import webdriver # 셀레니움
from numpy import NaN, NAN, nan # 결측치
from selenium.webdriver.edge.service import Service # 접근제한 우회
from selenium.webdriver.edge.options import Options # 접근제한 우회
from selenium.webdriver.support.ui import WebDriverWait # 기다리기
from selenium.webdriver.support import expected_conditions as EC # 요소 뜰 때까지 대기
from selenium.common.exceptions import TimeoutException # 시간 예외처리
from datetime import datetime

# 기본 정보 찾기 함수
def find_text(col, url):
    try:
        element = driver.find_element(By.CSS_SELECTOR, url).text
    except:
        element = '정보없음'
    col.append(element)

# 빈데이터프레임, 클래스
info_df = pd.DataFrame()
review_df = pd.DataFrame()
merged_df = pd.DataFrame()

# Edge WebDriver의 경로를 지정합니다. '경로/edgedriver'를 실제 경로로 교체해주세요.
service = Service(executable_path='C:\\selenium\\msedgedriver.exe')
options = webdriver.EdgeOptions()
options.use_chromium = True  # Chromium 기반 Edge 사용을 명시합니다.
# 사용자 에이전트 문자열을 설정합니다.
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36"
options.add_argument(f'user-agent={user_agent}')

# Edge 드라이버를 초기화합니다.
driver = webdriver.Edge(service=service, options=options)

# 구글 맵 들어가기
url = f'https://www.google.co.kr/maps/?hl=ko&entry=ttu'
driver.get(url)

time.sleep(random.uniform(2,3))
search_box = driver.find_element(By.XPATH, '//*[@id="searchboxinput"]')
search_box.send_keys("서울특별시 마포구 빵")
search_box.send_keys(Keys.ENTER)

i = 3  # [step1] 인포, 리뷰, "클릭" 순으로 변경한다.
while True:
    time.sleep(random.uniform(3,4))  # 구글 들어가서 대기
    driver.find_element(By.XPATH, f'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{i}]/div/a').click()

    # 주의하자. 반복되는 건 전역변수/지역변수의 문제다.
    name = []
    rating = []
    review = []
    category = []
    adress = []

    time.sleep(random.uniform(2,3))
    find_text(name, ".DUwDvf.lfPIob") # 이름에서 막히면 restart하세요.
    find_text(rating, ".F7nice > span:nth-child(1)")
    find_text(review, ".F7nice > span:nth-child(2)")
    find_text(category, ".fontBodyMedium span button.DkEaL")
    find_text(adress, ".rogA2c")

    # 임시 데이터프레임 (concat된 후 리셋됨)
    time.sleep(random.uniform(2,3))
    tmp = pd.DataFrame()
    tmp['Store'] = name
    tmp['Review_score'] = rating
    tmp['Review_counts'] = review
    tmp['Category'] = category
    tmp['Address'] = adress

    # 최종 데이터프레임은 전역변수에 들어감 > 빈 DF에 덮어쓰기
    time.sleep(random.uniform(2,3))
    info_df = pd.concat([info_df, tmp])
    
    # [step2] 리뷰 버튼 클릭 전 대기 [청수당, 아티장 에러주의]
    # time.sleep(random.uniform(3,5))
    # driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[3]/div/div/button[2]').click()

    try:
        # WebDriverWait과 expected_conditions를 사용하여 요소가 나타날 때까지 최대 10초간 대기
        elements = WebDriverWait(driver, 100).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[3]/div/div/button[2]')))
        for element in elements:
            element.click()  # 요소가 발견되면 클릭
    # except TimeoutException:
    #     print("해당 요소를 찾는 데 시간이 너무 오래 걸립니다.")
    except:
        pass # 리뷰 버튼도 정보 버튼도 없는 상태

    # element = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[3]/div/div/button[2]')
    # driver.execute_script("arguments[0].click();", element)

    # 정렬 버튼 클릭하기 전 대기
    try:
        result = []  # 새로운 리뷰만 넣을 건데, 중복을 제거하는 조건을 넣어야 한다.
        time_list = []

        time.sleep(random.uniform(3,4))
        driver.find_element(By.CSS_SELECTOR, 'div.m6QErb.Pf6ghf.KoSBEe.ecceSd.tLjsW > div.TrU0dc.kdfrQc').click()

        # 최신순 클릭하기 전 대기
        time.sleep(random.uniform(1,3))
        driver.find_element(By.XPATH, '//*[@id="action-menu"]/div[2]').click()

        # result = []  # 새로운 리뷰만 넣을 건데, 중복을 제거하는 조건을 넣어야 한다.
        # time_list = []

        time.sleep(random.uniform(2,4))  # 클릭하고 대기시간 필요하다. [에러주의]
        scroll = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[3]')
        last_height = driver.execute_script("return arguments[0].scrollHeight", scroll)
        while True:
            driver.execute_script("arguments[0].scrollTo(0, 300);", scroll)
            time.sleep(random.uniform(3, 4))  # 스크롤 후 잠시 대기
            buttons = driver.find_elements(By.CSS_SELECTOR, ".w8nwRe.kyuRq")
            for button in buttons:
                if button.is_displayed():
                    if button.is_enabled():
                        # 버튼이 활성화되어 있으면 클릭
                        button.click()
                    print("버튼을 클릭했습니다.")
                    break  # 첫 번째 활성화된 버튼을 클릭한 후 반복을 멈춤
                else:
                    print("활성화된 버튼이 없습니다.")
            time.sleep(random.uniform(3, 4))
            new_height = driver.execute_script("return arguments[0].scrollHeight", scroll)
            if new_height == last_height:
                break
            last_height = new_height

        last_review = None  # 비교하는 변수를 추가한다. WOW
        flag = False  # 종료 플래그를 사용해본다.
        while not flag:
            time.sleep(random.uniform(2, 3))  # 페이지 로드를 기다린다.
            element = driver.find_element(By.CSS_SELECTOR, '#QA0Szd')  # 소름 돋는 점. 필요 없는 코드다.
            reviews = driver.find_elements(By.CSS_SELECTOR, '.wiI7pd')  # 현재 로드된 모든 리뷰를 찾는다. elements로 가져와서 len=1의 리스트가 됐다.
            
            if not reviews:  # 강북상회(1) 리뷰가 없다면 루프를 종료한다.
                break

            for review in reviews:
                review_text = review.text
                if review_text == last_review:  # 마지막으로 확인한 리뷰와 같다면 새 리뷰 로드를 기다린다.
                    break
                if review_text not in result:
                    if review.tag_name.lower() == 'span':
                        result.append(review_text)
                        now = datetime.now().strftime('%Y-%m-%d - %H:%M:%S')
                        time_list.append(now)
                else:
                    flag=True
            
            if len(result) == 1:  # 로드된 리뷰가 하나만 있다면, 더 이상 새 리뷰가 없는 것으로 간주하고 루프를 종료한다.
                break
                
            last_review = result[-1] if result else None  # 마지막 리뷰를 확인용 변수에 업데이트 한다. result가 빈값으면(False) None을 쓴다.
        
        # 강북빵상회(2)
        if result:
            # 리뷰 데이터프레임을 만든다.
            tmp2 = pd.DataFrame()
            tmp2['Store'] = name*len(result)
            tmp2['Address'] = adress*len(result)
            tmp2['Review_text'] = result
            tmp2['Time'] = time_list

            # 리뷰 데이터프레임을 적재한다.
            time.sleep(random.uniform(1,2))
            review_df = pd.concat([review_df, tmp2])
    except:
        pass  # 정렬 버튼이 없으면 지나가시오.

    # [step3] 다음 가게 번호 부여하고 
    time.sleep(random.uniform(1,2))
    try:
        i+=2
        element = driver.find_element(By.XPATH, f'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{i}]/div/a')
        driver.execute_script("arguments[0].scrollIntoView(true);", element) # element 위치로 이동 > 수집 반복
    except:
        break

    print(f"{name}상점에서 {len(result)}개의 리뷰를 수집했습니다.")  # 파머스반브래드 12개> 28개

# 통합 데이터프레임
merged_df = pd.merge(info_df, review_df, on=["Store", "Address"], how="inner")
merged_df.to_csv('google_Mapo.csv', index=False)

버튼을 클릭했습니다.
버튼을 클릭했습니다.
버튼을 클릭했습니다.
버튼을 클릭했습니다.
버튼을 클릭했습니다.
버튼을 클릭했습니다.
버튼을 클릭했습니다.
버튼을 클릭했습니다.
버튼을 클릭했습니다.
버튼을 클릭했습니다.
활성화된 버튼이 없습니다.
['파네트']상점에서 7개의 리뷰를 수집했습니다.
버튼을 클릭했습니다.
버튼을 클릭했습니다.
버튼을 클릭했습니다.
버튼을 클릭했습니다.


#### 20. [성공] 적재 시점 추가 > 여기에 자세히 버튼 눌러야 해> 위코드로 이동

In [1]:
import pandas as pd # 판다스
import time # 시간차로 인한 에러방지
import random
from tqdm import tqdm # 코드가 멋져짐
from selenium.webdriver.common.by import By # 경로추적
from selenium.webdriver.common.keys import Keys # 엔터키
from selenium import webdriver # 셀레니움
from numpy import NaN, NAN, nan # 결측치
from selenium.webdriver.edge.service import Service # 접근제한 우회
from selenium.webdriver.edge.options import Options # 접근제한 우회
from selenium.webdriver.support.ui import WebDriverWait # 기다리기
from selenium.webdriver.support import expected_conditions as EC # 요소 뜰 때까지 대기
from selenium.common.exceptions import TimeoutException # 시간 예외처리
from datetime import datetime

# 기본 정보 찾기 함수
def find_text(col, url):
    try:
        element = driver.find_element(By.CSS_SELECTOR, url).text
    except:
        element = '정보없음'
    col.append(element)

# 빈데이터프레임, 클래스
info_df = pd.DataFrame()
review_df = pd.DataFrame()
merged_df = pd.DataFrame()

# Edge WebDriver의 경로를 지정합니다. '경로/edgedriver'를 실제 경로로 교체해주세요.
service = Service(executable_path='C:\\selenium\\msedgedriver.exe')
options = webdriver.EdgeOptions()
options.use_chromium = True  # Chromium 기반 Edge 사용을 명시합니다.
# 사용자 에이전트 문자열을 설정합니다.
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36"
options.add_argument(f'user-agent={user_agent}')

# Edge 드라이버를 초기화합니다.
driver = webdriver.Edge(service=service, options=options)

# 구글 맵 들어가기
url = f'https://www.google.co.kr/maps/?hl=ko&entry=ttu'
driver.get(url)

time.sleep(random.uniform(2,3))
search_box = driver.find_element(By.XPATH, '//*[@id="searchboxinput"]')
search_box.send_keys("서울특별시 마포구 빵")
search_box.send_keys(Keys.ENTER)

i = 3  # [step1] 인포, 리뷰, "클릭" 순으로 변경한다.
while True:
    time.sleep(random.uniform(3,4))  # 구글 들어가서 대기
    driver.find_element(By.XPATH, f'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{i}]/div/a').click()

    # 주의하자. 반복되는 건 전역변수/지역변수의 문제다.
    name = []
    rating = []
    review = []
    category = []
    adress = []

    time.sleep(random.uniform(2,3))
    find_text(name, ".DUwDvf.lfPIob") # 이름에서 막히면 restart하세요.
    find_text(rating, ".F7nice > span:nth-child(1)")
    find_text(review, ".F7nice > span:nth-child(2)")
    find_text(category, ".fontBodyMedium span button.DkEaL")
    find_text(adress, ".rogA2c")

    # 임시 데이터프레임 (concat된 후 리셋됨)
    time.sleep(random.uniform(2,3))
    tmp = pd.DataFrame()
    tmp['Store'] = name
    tmp['Review_score'] = rating
    tmp['Review_counts'] = review
    tmp['Category'] = category
    tmp['Address'] = adress

    # 최종 데이터프레임은 전역변수에 들어감 > 빈 DF에 덮어쓰기
    time.sleep(random.uniform(2,3))
    info_df = pd.concat([info_df, tmp])
    
    # [step2] 리뷰 버튼 클릭 전 대기 [청수당, 아티장 에러주의]
    # time.sleep(random.uniform(3,5))
    # driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[3]/div/div/button[2]').click()

    try:
        # WebDriverWait과 expected_conditions를 사용하여 요소가 나타날 때까지 최대 10초간 대기
        elements = WebDriverWait(driver, 100).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[3]/div/div/button[2]')))
        for element in elements:
            element.click()  # 요소가 발견되면 클릭
    # except TimeoutException:
    #     print("해당 요소를 찾는 데 시간이 너무 오래 걸립니다.")
    except:
        pass # 리뷰 버튼도 정보 버튼도 없는 상태

    # element = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[3]/div/div/button[2]')
    # driver.execute_script("arguments[0].click();", element)

    # 정렬 버튼 클릭하기 전 대기
    try:
        result = []  # 새로운 리뷰만 넣을 건데, 중복을 제거하는 조건을 넣어야 한다.
        time_list = []

        time.sleep(random.uniform(3,4))
        driver.find_element(By.CSS_SELECTOR, 'div.m6QErb.Pf6ghf.KoSBEe.ecceSd.tLjsW > div.TrU0dc.kdfrQc').click()

        # 최신순 클릭하기 전 대기
        time.sleep(random.uniform(1,3))
        driver.find_element(By.XPATH, '//*[@id="action-menu"]/div[2]').click()

        # result = []  # 새로운 리뷰만 넣을 건데, 중복을 제거하는 조건을 넣어야 한다.
        # time_list = []

        time.sleep(random.uniform(2,4))  # 클릭하고 대기시간 필요하다. [에러주의]
        scroll = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[3]')
        last_height = driver.execute_script("return arguments[0].scrollHeight", scroll)
        while True:
            driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", scroll)
            time.sleep(random.uniform(3, 4))  # 새 리뷰가 로드되기를 기다린다.
            new_height = driver.execute_script("return arguments[0].scrollHeight", scroll)
            if new_height == last_height:
                break
            last_height = new_height

        last_review = None  # 비교하는 변수를 추가한다. WOW
        flag = False  # 종료 플래그를 사용해본다.
        while not flag:
            time.sleep(random.uniform(2, 3))  # 페이지 로드를 기다린다.
            element = driver.find_element(By.CSS_SELECTOR, '#QA0Szd')  # 소름 돋는 점. 필요 없는 코드다.
            reviews = driver.find_elements(By.CSS_SELECTOR, '.wiI7pd')  # 현재 로드된 모든 리뷰를 찾는다. elements로 가져와서 len=1의 리스트가 됐다.
            
            if not reviews:  # 강북상회(1) 리뷰가 없다면 루프를 종료한다.
                break

            for review in reviews:
                review_text = review.text
                if review_text == last_review:  # 마지막으로 확인한 리뷰와 같다면 새 리뷰 로드를 기다린다.
                    break
                if review_text not in result:
                    if review.tag_name.lower() == 'span':
                        result.append(review_text)
                        now = datetime.now().strftime('%Y-%m-%d - %H:%M:%S')
                        time_list.append(now)
                else:
                    flag=True
            
            if len(result) == 1:  # 로드된 리뷰가 하나만 있다면, 더 이상 새 리뷰가 없는 것으로 간주하고 루프를 종료한다.
                break
                
            last_review = result[-1] if result else None  # 마지막 리뷰를 확인용 변수에 업데이트 한다. result가 빈값으면(False) None을 쓴다.
        
        # 강북빵상회(2)
        if result:
            # 리뷰 데이터프레임을 만든다.
            tmp2 = pd.DataFrame()
            tmp2['Store'] = name*len(result)
            tmp2['Address'] = adress*len(result)
            tmp2['Review_text'] = result
            tmp2['Time'] = time_list

            # 리뷰 데이터프레임을 적재한다.
            time.sleep(random.uniform(1,2))
            review_df = pd.concat([review_df, tmp2])
    except:
        pass  # 정렬 버튼이 없으면 지나가시오.

    # [step3] 다음 가게 번호 부여하고 
    time.sleep(random.uniform(1,2))
    try:
        i+=2
        element = driver.find_element(By.XPATH, f'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{i}]/div/a')
        driver.execute_script("arguments[0].scrollIntoView(true);", element) # element 위치로 이동 > 수집 반복
    except:
        break

    print(f"{name}상점에서 {len(result)}개의 리뷰를 수집했습니다.")  # 파머스반브래드 12개> 28개

# 통합 데이터프레임
merged_df = pd.merge(info_df, review_df, on=["Store", "Address"], how="inner")
merged_df.to_csv('google_Mapo.csv', index=False)

['만동제과']상점에서 257개의 리뷰를 수집했습니다.
['곳간']상점에서 9개의 리뷰를 수집했습니다.
['파파브레드']상점에서 50개의 리뷰를 수집했습니다.
['독일빵집']상점에서 20개의 리뷰를 수집했습니다.
['폴앤폴리나 연희점']상점에서 92개의 리뷰를 수집했습니다.
['바타르']상점에서 1개의 리뷰를 수집했습니다.
['성북당 십원빵 연세대점']상점에서 0개의 리뷰를 수집했습니다.
['유일한베이커리']상점에서 15개의 리뷰를 수집했습니다.
['이상미통밀빵']상점에서 19개의 리뷰를 수집했습니다.
['뉘블랑쉬']상점에서 18개의 리뷰를 수집했습니다.
['뺑드에떼']상점에서 3개의 리뷰를 수집했습니다.
['에브리코너바이트']상점에서 4개의 리뷰를 수집했습니다.
['피터팬 1978']상점에서 455개의 리뷰를 수집했습니다.
['파리바게뜨 서대문센트레빌']상점에서 6개의 리뷰를 수집했습니다.
['피터팬 사러가점']상점에서 4개의 리뷰를 수집했습니다.
['주재근베이커리']상점에서 76개의 리뷰를 수집했습니다.
['고르드 Gordes Bakery Cafe']상점에서 168개의 리뷰를 수집했습니다.
['동네빵네 마실']상점에서 2개의 리뷰를 수집했습니다.
['파티세리 온고 (Ongo)']상점에서 15개의 리뷰를 수집했습니다.
['사워도우 랩']상점에서 13개의 리뷰를 수집했습니다.
['알레스구떼']상점에서 39개의 리뷰를 수집했습니다.
['글룩베이커리']상점에서 5개의 리뷰를 수집했습니다.
['뺑라팡']상점에서 4개의 리뷰를 수집했습니다.
['올도우베이커리']상점에서 2개의 리뷰를 수집했습니다.
['퍼스트브래드 북가좌점']상점에서 1개의 리뷰를 수집했습니다.
['파리바게뜨 서대문행복점']상점에서 22개의 리뷰를 수집했습니다.
['호호빵집']상점에서 1개의 리뷰를 수집했습니다.
['오르랔베이커리']상점에서 22개의 리뷰를 수집했습니다.
['수신당']상점에서 8개의 리뷰를 수집했습니다.
['파랑베이커리']상점에서 10개의 리뷰를 수집했습니다.
['디저트콩콩']상점에서 

In [ ]:
# 적재시점 추가하기 코드
from datetime import datetime
time_list = []
now = datetime.now().strftime('%Y-%m-%d - %H:%M:%S')
time_list.append(now)

#### 19. [와이파이 이슈 빼고/성공] 작성자, 작성일자 추가한 전체코드(point:tag_name.lower())

In [1]:
import pandas as pd # 판다스
import time # 시간차로 인한 에러방지
import random
from tqdm import tqdm # 코드가 멋져짐
from selenium.webdriver.common.by import By # 경로추적
from selenium.webdriver.common.keys import Keys # 엔터키
from selenium import webdriver # 셀레니움
from numpy import NaN, NAN, nan # 결측치
from selenium.webdriver.edge.service import Service # 접근제한 우회
from selenium.webdriver.edge.options import Options # 접근제한 우회
from selenium.webdriver.support.ui import WebDriverWait # 기다리기
from selenium.webdriver.support import expected_conditions as EC # 요소 뜰 때까지 대기
from selenium.common.exceptions import TimeoutException # 시간 예외처리

# 기본 정보 찾기 함수
def find_text(col, url):
    try:
        element = driver.find_element(By.CSS_SELECTOR, url).text
    except:
        element = '정보없음'
    col.append(element)

# 빈데이터프레임, 클래스
info_df = pd.DataFrame()
review_df = pd.DataFrame()
merged_df = pd.DataFrame()

# Edge WebDriver의 경로를 지정합니다. '경로/edgedriver'를 실제 경로로 교체해주세요.
service = Service(executable_path='C:\\selenium\\msedgedriver.exe')
options = webdriver.EdgeOptions()
options.use_chromium = True  # Chromium 기반 Edge 사용을 명시합니다.
# 사용자 에이전트 문자열을 설정합니다.
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36"
options.add_argument(f'user-agent={user_agent}')

# Edge 드라이버를 초기화합니다.
driver = webdriver.Edge(service=service, options=options)

# 구글 맵 들어가기
url = f'https://www.google.co.kr/maps/?hl=ko&entry=ttu'
driver.get(url)

time.sleep(random.uniform(2,3))
search_box = driver.find_element(By.XPATH, '//*[@id="searchboxinput"]')
search_box.send_keys("서울특별시 종로구 빵")
search_box.send_keys(Keys.ENTER)

i = 3  # [step1] 인포, 리뷰, "클릭" 순으로 변경한다.
while True:
    time.sleep(random.uniform(3,4))  # 구글 들어가서 대기
    driver.find_element(By.XPATH, f'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{i}]/div/a').click()

    # 주의하자. 반복되는 건 전역변수/지역변수의 문제다.
    name = []
    rating = []
    review = []
    category = []
    adress = []

    time.sleep(random.uniform(2,3))
    find_text(name, ".DUwDvf.lfPIob") # 이름에서 막히면 restart하세요.
    find_text(rating, ".F7nice > span:nth-child(1)")
    find_text(review, ".F7nice > span:nth-child(2)")
    find_text(category, ".fontBodyMedium span button.DkEaL")
    find_text(adress, ".rogA2c")

    # 임시 데이터프레임 (concat된 후 리셋됨)
    time.sleep(random.uniform(2,3))
    tmp = pd.DataFrame()
    tmp['name'] = name
    tmp['rating'] = rating
    tmp['review'] = review
    tmp['category'] = category
    tmp['adress'] = adress

    # 최종 데이터프레임은 전역변수에 들어감 > 빈 DF에 덮어쓰기
    time.sleep(random.uniform(2,3))
    info_df = pd.concat([info_df, tmp])
    
    # [step2] 리뷰 버튼 클릭 전 대기 [청수당, 아티장 에러주의]
    # time.sleep(random.uniform(3,5))
    # driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[3]/div/div/button[2]').click()

    try:
        # WebDriverWait과 expected_conditions를 사용하여 요소가 나타날 때까지 최대 10초간 대기
        elements = WebDriverWait(driver, 60).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[3]/div/div/button[2]')))
        for element in elements:
            element.click()  # 요소가 발견되면 클릭
    # except TimeoutException:
    #     print("해당 요소를 찾는 데 시간이 너무 오래 걸립니다.")
    except:
        time.sleep(random.uniform(3,5))
        driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[3]/div/div/button[2]').click()

    # element = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[3]/div/div/button[2]')
    # driver.execute_script("arguments[0].click();", element)

    # 정렬 버튼 클릭하기 전 대기
    try:
        time.sleep(random.uniform(3,4))
        driver.find_element(By.CSS_SELECTOR, 'div.m6QErb.Pf6ghf.KoSBEe.ecceSd.tLjsW > div.TrU0dc.kdfrQc').click()

        # 최신순 클릭하기 전 대기
        time.sleep(random.uniform(1,3))
        driver.find_element(By.XPATH, '//*[@id="action-menu"]/div[2]').click()

        time.sleep(random.uniform(2,4))  # 클릭하고 대기시간 필요하다. [에러주의]
        scroll = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[3]')
        last_height = driver.execute_script("return arguments[0].scrollHeight", scroll)
        while True:
            driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", scroll)
            time.sleep(random.uniform(3, 4))  # 새 리뷰가 로드되기를 기다린다.
            new_height = driver.execute_script("return arguments[0].scrollHeight", scroll)
            if new_height == last_height:
                break
            last_height = new_height

        time.sleep(random.uniform(5,10))
        rev_elements = driver.find_elements(By.CLASS_NAME, "GHT2ce")
        
        writer = []
        redate = []
        review = []
        result_df = pd.DataFrame()

        # 작성자d4r55 작성일rsqaWe 리뷰wiI7pd
        count=0
        for element in rev_elements:
            if count%2 == 1:
                element4 = element.find_element(By.CLASS_NAME, "rsqaWe").text
                redate.append(element4)
                try:
                    element2 = element.find_element(By.CLASS_NAME, "wiI7pd")
                    if element2.tag_name.lower() == 'span':
                        review.append(element2.text)
                except:
                    review.append("정보없음")
            else:
                element3 = element.find_element(By.CLASS_NAME, "d4r55").text
                writer.append(element3)
            count+=1

        # 리뷰 데이터프레임을 만든다.
        tmp2 = pd.DataFrame()
        tmp2['name'] = name*len(review)
        tmp2['writer'] = writer
        tmp2['redate'] = redate
        tmp2['review'] = review

        # 리뷰 데이터프레임을 적재한다.
        time.sleep(random.uniform(1,2))
        review_df = pd.concat([review_df, tmp2])

    except:
        # result = []  # 리뷰0개 프린트용으로 추가 후 에러나서 제거
        pass  # 정렬 버튼이 없으면 지나가시오.

    # [step3] 다음 가게 번호 부여하고 
    time.sleep(random.uniform(1,2))
    try:
        i+=2
        element = driver.find_element(By.XPATH, f'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{i}]/div/a')
        driver.execute_script("arguments[0].scrollIntoView(true);", element) # element 위치로 이동 > 수집 반복
    except:
        break
    
    # if review == ['정보없음']:  # 리뷰0개 프린트용으로 추가 후 에러나서 제거
    #     result = []
    print(f"{name}상점에서 {len(review)}개의 리뷰를 수집했습니다.")  # 파머스반브래드 12개> 28개

['파머스반브레드']상점에서 14개의 리뷰를 수집했습니다.
['뚜레쥬르 종로점']상점에서 10개의 리뷰를 수집했습니다.
['쁘띠통']상점에서 65개의 리뷰를 수집했습니다.
['솔트24']상점에서 110개의 리뷰를 수집했습니다.
['종로명인단팥빵']상점에서 10개의 리뷰를 수집했습니다.
['안국 153']상점에서 295개의 리뷰를 수집했습니다.
['솔방울베이커리']상점에서 56개의 리뷰를 수집했습니다.
['삼평빵집']상점에서 2개의 리뷰를 수집했습니다.
['사베 종각']상점에서 13개의 리뷰를 수집했습니다.
['김용현베이커리']상점에서 57개의 리뷰를 수집했습니다.
['체부동다락빵']상점에서 19개의 리뷰를 수집했습니다.
['파네트 크루아상팩토리']상점에서 58개의 리뷰를 수집했습니다.
['건강한빵']상점에서 39개의 리뷰를 수집했습니다.
['가드니아 제빵소 종로타워점']상점에서 1개의 리뷰를 수집했습니다.
['강북빵상회']상점에서 1개의 리뷰를 수집했습니다.
['아베베베이커리 서울']상점에서 87개의 리뷰를 수집했습니다.
['효자베이커리']상점에서 402개의 리뷰를 수집했습니다.
['자연도소금빵 & 자연도가']상점에서 67개의 리뷰를 수집했습니다.
['엠마']상점에서 56개의 리뷰를 수집했습니다.
['앞으로의 빵집']상점에서 58개의 리뷰를 수집했습니다.
['허밍벨라베이커리']상점에서 1개의 리뷰를 수집했습니다.
['우즈베이커리 익선점']상점에서 83개의 리뷰를 수집했습니다.
['토리']상점에서 27개의 리뷰를 수집했습니다.
['황주명새싹베이커리']상점에서 1개의 리뷰를 수집했습니다.
['엘리스파이']상점에서 4개의 리뷰를 수집했습니다.
['묘한빵집']상점에서 23개의 리뷰를 수집했습니다.
['종유당']상점에서 4개의 리뷰를 수집했습니다.
['몽핀']상점에서 188개의 리뷰를 수집했습니다.
['황금똥빵 종로본점']상점에서 2개의 리뷰를 수집했습니다.
['빅토리아베이커리 서촌']상점에서 25개의 리뷰를 수집했습니다.
['하이버']상점에서 68개의 리뷰를 수집했습

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[3]/div/div/button[2]"}
  (Session info: MicrosoftEdge=123.0.2420.65); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7E560B852+63538]
	Microsoft::Applications::Events::ILogConfiguration::operator* [0x00007FF7E558F6D2+238290]
	(No symbol) [0x00007FF7E53C3407]
	(No symbol) [0x00007FF7E54097ED]
	(No symbol) [0x00007FF7E54098B0]
	(No symbol) [0x00007FF7E5444387]
	(No symbol) [0x00007FF7E5428E5F]
	(No symbol) [0x00007FF7E53FF037]
	(No symbol) [0x00007FF7E5441F49]
	(No symbol) [0x00007FF7E5428A83]
	(No symbol) [0x00007FF7E53FE55E]
	(No symbol) [0x00007FF7E53FD71C]
	(No symbol) [0x00007FF7E53FE111]
	Microsoft::Applications::Events::EventProperty::to_string [0x00007FF7E57E1D3C+1102764

In [3]:
info_df

,name,rating,review,category,adress
0,파머스반브레드,5.0,(14),제과점,서울특별시 종로구 돈화문로11가길 59 현대뜨레비앙 111호
0,솔방울베이커리,4.5,(56),제과점,서울특별시 종로구 서순라길 89-15
0,종로명인단팥빵,4.1,(10),제과점,서울특별시 종로구 수표로 98
0,종로명인단팥빵,4.1,(10),제과점,서울특별시 종로구 수표로 98


In [4]:
review_df

,name,writer,redate,review
0,파머스반브레드,David Sielmann,3달 전,정보없음
1,파머스반브레드,Wilbert Aguilar,5달 전,정말 맛있는 유럽빵. 아몬드 크로아상 정말 맛있어요! 아몬드 페이스트가 가득 들어있...
2,파머스반브레드,Ulrich Link,8달 전,"독일보다 나은 서울의 비트 베이커리, 버터가 들어간 브레첼은 꼭 먹어보세요, 꼭 먹..."
3,파머스반브레드,JIYEON LEE,10달 전,이 곳 빵은 좋은 제품들로 만들어져서 비싸지만 정말 그 값을 합니다! 많이 먹어도 ...
4,파머스반브레드,Karn,10달 전,"훌륭한 음료 선택, 차분한 분위기. 케이크 선택이 하이라이트입니다. 인테리어도 너무..."
...,...,...,...,...
51,솔방울베이커리,Allisen Kang강승희,1년 전,시원하고 마음의 휴식이 있는 곳
52,솔방울베이커리,jlnyxx y,1년 전,사랑스러운 한옥 스타일의 카페. 훌륭한 커피와 패스트리. 햄 & 치즈 크루아상은 완...
53,솔방울베이커리,Dominic Lim,1년 전,좋은 분위기의 훌륭한 카페! 장소는 상당히 새로운 것 같습니다. 치즈 크로아상을 적...
54,솔방울베이커리,S.W. SONG,1년 전,한옥구조의 베이커리 카페.\n빵집인줄 알았는데 안쪽으로도 공간이 꽤 넓고\n2층에도...


#### 18. 작성일, 작성자 간단하게 테스트 먼저하고 다음으로

In [1]:
import pandas as pd # 판다스
import time # 시간차로 인한 에러방지
import random # 시간차로 인한 에러방지
import requests # 웹스크래핑
from tqdm import tqdm # 코드가 멋져짐
from selenium.webdriver.common.by import By # 경로추적
from selenium import webdriver # 셀레니움
from bs4 import BeautifulSoup # 뷰숲

driver = webdriver.Edge()

# "서울특별시 종로구 빵" 검색 페이지 바로 들어가기(=동적 페이지)
url = f'https://www.google.co.kr/maps/search/%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C+%EC%A2%85%EB%A1%9C%EA%B5%AC+%EB%B9%B5/data=!3m1!4b1?hl=ko&entry=ttu'
driver.get(url)

# n번째_빵집 클릭
time.sleep(random.uniform(1,3))
driver.find_element(By.XPATH, f'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{5}]/div/a').click()

# 이름 수집
time.sleep(random.uniform(1,3))
element = driver.find_element(By.CSS_SELECTOR, ".DUwDvf.lfPIob")

# 리뷰 버튼 클릭
time.sleep(random.uniform(1,3))
driver.find_element(By.CSS_SELECTOR, 'button:nth-child(2)').click()

# 정렬 버튼 클릭하기
time.sleep(random.uniform(1,3))
driver.find_element(By.CSS_SELECTOR, 'div.m6QErb.Pf6ghf.KoSBEe.ecceSd.tLjsW > div.TrU0dc.kdfrQc').click()

# 최신순 클릭하기
time.sleep(random.uniform(1,3))
driver.find_element(By.XPATH, '//*[@id="action-menu"]/div[2]').click()

# 스크롤 추가
time.sleep(random.uniform(2,4))  # 클릭하고 대기시간 필요하다. [에러주의]
scroll = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[3]')
last_height = driver.execute_script("return arguments[0].scrollHeight", scroll)
while True:
    driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", scroll)
    time.sleep(random.uniform(3, 4))  # 새 리뷰가 로드되기를 기다린다.
    new_height = driver.execute_script("return arguments[0].scrollHeight", scroll)
    if new_height == last_height:
        break
    last_height = new_height

# 리스트 초기화
writer = []
writer_date = []
writer_review = []

# 리뷰 수집 wiI7pd
elements_1 = driver.find_elements(By.CLASS_NAME, "GHT2ce")
print(len(elements_1))
# 작성자 수집 d4r55-div
elements_2 = driver.find_elements(By.CLASS_NAME, "d4r55")  # elements_1에 속한다.
print(len(elements_2))
# 작성일 수집 rsqaWe-span
elements_3 = driver.find_elements(By.CLASS_NAME, "rsqaWe")  # elements_1에 속한다.
print(len(elements_3))

# # 찾은 요소들 출력
# for review_element in elements:
#     try:
#         # 리뷰 텍스트 추출 시도 (천잰데?)
#         tmp_review_element = review_element.find_element(By.CLASS_NAME, "wiI7pd")
#         tmp_review = tmp_review_element.find_element(By.TAG_NAME, 'span').text

#         if tmp_review:
#             tmp_writer = review_element.find_element(By.CLASS_NAME, "d4r55").text
#             tmp_date = review_element.find_element(By.CLASS_NAME, "rsqaWe").text

#         # 리스트에 추가
#         writer.append(tmp_writer)
#         writer_date.append(tmp_date)
#         writer_review.append(tmp_review)

#         # 결과 출력
#         print(tmp_writer)
#         print(tmp_date)
#         print(tmp_review)

#     except Exception as e:
#         # 리뷰 텍스트 추출에 실패한 경우, 오류 메시지 출력
#         print(f"리뷰 텍스트 추출 실패 이유: {e}")
#         continue

130
65
65


In [ ]:
# 65/65/65여야 인덱스를 맞출 수 있다. 130이 모두를 포함하며 2줄씩 있어서 사실 나누면 65/65/65가 맞다.

In [43]:
# 짝수번째 작성자
elements_1[0].text # 목표: 65개, 65개, 65개 -> 65*2 = 130개를 가지고 전부 진행 중

'Myriam Phung\n지역 가이드 · 리뷰 486개 · 사진 1,194장\n\ue5d4'

In [9]:
# 홀수번째 작성일, 리뷰
elements_1[117].text # 목표: 65개, 65개, 65개 -> 65*2 = 130개를 가지고 전부 진행 중

'\ue838\n\ue838\n\ue838\n\ue838\n\ue838\n5년 전\n샌드위치에 내용물을 듬뿍 넣어서 한끼 식사로 좋아요. 빵들도 맛있어요.\n\ue8dc\n좋아요\n\ue80d\n공유\n업체 대표 응답 5년 전\n감사합니다^^'

In [46]:
for i in elements_1:
    try:
        print(i.find_element(By.CLASS_NAME, "d4r55").text)
    except:
        print("정보없음")

Myriam Phung
정보없음
박시현
정보없음
Ola Sol
정보없음
Mr. Son
정보없음
Jen Chon
정보없음
츄릅
정보없음
scully
정보없음
이기적
정보없음
감천
정보없음
say cheese
정보없음
Tae Young Lee
정보없음
Jörg Blank
정보없음
Jin Soo Kim
정보없음
Jin Mudra
정보없음
oh
정보없음
Gunjeong Lee
정보없음
안로즈
정보없음
길영란
정보없음
taeyoung lee
정보없음
keeper
정보없음
Cheolhee Cho
정보없음
opry
정보없음
유유
정보없음
경복궁한복대여해와달한복 대여/日月韩服
정보없음
Kim Chanhee
정보없음
이태훈
정보없음
Junghoon Han
정보없음
Byeongjo Hwangbo
정보없음
Hyun Chung
정보없음
하늘유성
정보없음
Miae Lee
정보없음
Do
정보없음
Kwangsoo Sun
정보없음
김남윤
정보없음
경성민
정보없음
Yunchon Kim
정보없음
누상인
정보없음
백두산
정보없음
yoonjung chae
정보없음
Lee Seung Won
정보없음
김효순
정보없음
OArtism
정보없음
sodam
정보없음
스시도쿠
정보없음
최찬돌 (찬돌이)
정보없음
Choco mi
정보없음
신찬길
정보없음
S로 사는 이야기
정보없음
Bar_ Tyon
정보없음
김영
정보없음
R. Park
정보없음
안재진
정보없음
Finish Mad
정보없음
김효진
정보없음
방구부루룩
정보없음
BoMin Kim
정보없음
Heinz Pfister
정보없음
Lee SY
정보없음
오형석
정보없음
haha hong
정보없음
Rosemary
정보없음
설리
정보없음
dahli A
정보없음
ramazzotti
정보없음
rednsky
정보없음


In [49]:
# 쓸모있는 최종 코드 (홀/짝 개념 활용)
review = []
writer = []
redate = []
result_df = pd.DataFrame()

# 작성자d4r55 작성일rsqaWe 리뷰wiI7pd
count=0
for element in elements_1:
    if count%2 == 1:
        element4 = element.find_element(By.CLASS_NAME, "rsqaWe").text
        redate.append(element4)
        try:
            element2 = element.find_element(By.CLASS_NAME, "wiI7pd")
            if element2.tag_name.lower() == 'span':
                review.append(element2.text)
        except:
            review.append("정보없음")
    else:
        element3 = element.find_element(By.CLASS_NAME, "d4r55").text
        writer.append(element3)
    count+=1

tmp = pd.DataFrame()
tmp['writer'] = writer
tmp['redate'] = redate
tmp['review'] = review
result_df = pd.concat([result_df, tmp])
result_df

,writer,redate,review
0,Myriam Phung,3달 전,"집에서 만든 듯한 100% 호밀빵. 소박한 맛과 투박한 식감이 특징인데, 특출나진 ..."
1,박시현,5달 전,무화과 과즙 팡팡터지는 쁘띠통의 계절과일타르트 진짜 맛있어요🤍…
2,Ola Sol,5달 전,빵 및 다른 제품들 맛은 괜찮으나 가격이 높은 편이고 직원들이 무뚝뚝.
3,Mr. Son,7달 전,정보없음
4,Jen Chon,8달 전,"스위스, 프랑스, 독일에서 갓 구운 빵을 먹어본 적이 있는데, 그 빵의 품질에 매우..."
...,...,...,...
60,Rosemary,5년 전,연어 치아바타 3/3개. 차트에서 벗어난 샌드위치.
61,설리,5년 전,천연발효빵 비건샌드위치 타르트 굿
62,dahli A,6년 전,"비건들을 위한 빵, 유럽식 발효빵이 주에요\n스콘 , 프레젤 , 치아바타 모두 맛있..."
63,ramazzotti,6년 전,"스콘, 치아바타, 포카치아 요렇게 먹어봤는 데 맛있음"


In [2]:
# 리뷰 elements_1, 작성자 elements_2, 작성일 elements_3 함수 만들기
import pandas as pd

number = []
result = []

def find_review_list(elements):
    if elements == elements_1:
        for i, element in enumerate(elements):
            element_text = element.text
            if element.tag_name.lower() == 'span':
                number.append(i)
                result.append(element_text)
        tmp = pd.DataFrame()
        tmp['number'] = number
        tmp['result'] = result
        return tmp
    else:
        for i, element in enumerate(elements):
            element_text = element.text
            number.append(i)
            result.append(element_text)
        tmp = pd.DataFrame()
        tmp['number'] = number
        tmp['result'] = result
        return tmp

find_review_list(elements_1)  # 함수는 버려

,number,result


In [ ]:
# elements만 가지고서 리스트 추출 테스트 -> 이중 element는 왜인지 경로를 못찾음
for review_element in elements:
    try:
        # 리뷰 텍스트 추출 시도 (천잰데?)
        tmp_review_element = review_element.find_element(By.CLASS_NAME, "wiI7pd")
        tmp_review = tmp_review_element.find_element(By.TAG_NAME, 'span').text

        if tmp_review:
            tmp_writer = review_element.find_element(By.CLASS_NAME, "d4r55").text
            tmp_date = review_element.find_element(By.CLASS_NAME, "rsqaWe").text

        # 리스트에 추가
        writer.append(tmp_writer)
        writer_date.append(tmp_date)
        writer_review.append(tmp_review)

        # 결과 출력
        print(tmp_writer)
        print(tmp_date)
        print(tmp_review)

    except Exception as e:
        # 리뷰 텍스트 추출에 실패한 경우, 오류 메시지 출력
        print(f"리뷰 텍스트 추출 실패 이유: {e}")
        continue  # 버려

#### 17. 종로구 소프트케이크하우스로 실험 : 스크롤을 통과하는가? 이제 된다!

In [ ]:
import pandas as pd # 판다스
import time # 시간차로 인한 에러방지
import random
from tqdm import tqdm # 코드가 멋져짐
from selenium.webdriver.common.by import By # 경로추적
from selenium.webdriver.common.keys import Keys # 엔터키
from selenium import webdriver # 셀레니움
from numpy import NaN, NAN, nan # 결측치

# 기본 정보 찾기 함수
def find_text(col, url):
    try:
        element = driver.find_element(By.CSS_SELECTOR, url).text
    except:
        element = '정보없음'
    col.append(element)

# 빈데이터프레임, 클래스
info_df = pd.DataFrame()
review_df = pd.DataFrame()
merged_df = pd.DataFrame()
driver = webdriver.Edge()

# 구글 맵 들어가기
url = f'https://www.google.co.kr/maps/?hl=ko&entry=ttu'
driver.get(url)

search_box = driver.find_element(By.XPATH, '//*[@id="searchboxinput"]')
search_box.send_keys("서울특별시 소프트케이크하")
search_box.send_keys(Keys.ENTER)

i = 3  # [step1] 인포, 리뷰, "클릭" 순으로 변경한다.
time.sleep(random.uniform(3,4))
driver.find_element(By.XPATH, f'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{1}]/div/a').click()

# 주의하자. 반복되는 건 전역변수/지역변수의 문제다.
name = []
rating = []
review = []
category = []
adress = []

time.sleep(random.uniform(1,3))
find_text(name, ".DUwDvf.lfPIob") # 이름에서 막히면 restart하세요.
find_text(rating, ".F7nice > span:nth-child(1)")
find_text(review, ".F7nice > span:nth-child(2)")
find_text(category, ".fontBodyMedium span button.DkEaL")
find_text(adress, ".rogA2c")

# 임시 데이터프레임 (concat된 후 리셋됨)
time.sleep(random.uniform(1,3))
tmp = pd.DataFrame()
tmp['name'] = name
tmp['rating'] = rating
tmp['review'] = review
tmp['category'] = category
tmp['adress'] = adress

# 최종 데이터프레임은 전역변수에 들어감 > 빈 DF에 덮어쓰기
time.sleep(random.uniform(1,3))
info_df = pd.concat([info_df, tmp])

# [step2] 리뷰 버튼 클릭
time.sleep(random.uniform(3,4))
driver.find_element(By.CSS_SELECTOR, 'button:nth-child(2)').click()

# 정렬 버튼 클릭하기(=정보 누르는거 막기)
time.sleep(random.uniform(1,3))
driver.find_element(By.CSS_SELECTOR, 'div.m6QErb.Pf6ghf.KoSBEe.ecceSd.tLjsW > div.TrU0dc.kdfrQc').click()

# 최신순 클릭하기
time.sleep(random.uniform(1,3))
driver.find_element(By.XPATH, '//*[@id="action-menu"]/div[2]').click()

result = []  # 새로운 리뷰만 넣을 건데, 중복을 제거하는 조건을 넣어야 한다.
answer = []

time.sleep(random.uniform(2,3))  # 클릭하고 대기시간 필요하다. [에러주의]
scroll = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[3]')
last_height = driver.execute_script("return arguments[0].scrollHeight", scroll)
while True:
    driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", scroll)
    time.sleep(random.uniform(3, 4))  # 새 리뷰가 로드되기를 기다린다.
    new_height = driver.execute_script("return arguments[0].scrollHeight", scroll)
    if new_height == last_height:
        break
    last_height = new_height

print('1번째 통과입니다.')

last_review = None  # 비교하는 변수를 추가한다. WOW
flag = False  # 종료 플래그를 사용해본다.
while not flag:
    time.sleep(random.uniform(2, 3))  # 페이지 로드를 기다린다.
    element = driver.find_element(By.CSS_SELECTOR, '#QA0Szd')  # 소름 돋는 점. 필요 없는 코드다.
    reviews = driver.find_elements(By.CSS_SELECTOR, '.wiI7pd')  # 현재 로드된 모든 리뷰를 찾는다. elements로 가져와서 len=1의 리스트가 됐다.
    
    if not reviews:  # 리뷰가 없다면 루프를 종료한다.
        break

    for review in reviews:
        review_text = review.text
        if review_text == last_review:  # 마지막으로 확인한 리뷰와 같다면 새 리뷰 로드를 기다린다.
            break
        if review_text not in result:
            if review.tag_name.lower() == 'span':
                result.append(review_text)
            else:
                answer.append(review_text)
        else:
            flag=True

    if len(result) == 1:  # 로드된 리뷰가 하나만 있다면, 더 이상 새 리뷰가 없는 것으로 간주하고 루프를 종료한다.
        break
        
    last_review = result[-1] if result else None  # 마지막 리뷰를 확인용 변수에 업데이트 한다. result가 빈값으면(False) None을 쓴다.

print('2번째 통과입니다.')

# # 강북빵상회(2)
# if result:
#     # 리뷰 데이터프레임을 만든다.
#     tmp2 = pd.DataFrame()
#     tmp2['review'] = result
#     tmp2['name'] = name

#     # 리뷰 데이터프레임을 적재한다.
#     time.sleep(random.uniform(1,2))
#     review_df = pd.concat([review_df, tmp2])

#     # 머지 데이터프레임을 만들어서 적재한다.
#     time.sleep(random.uniform(1,2))
#     tmp3 = pd.merge(tmp, tmp2, on="name", how="right")
#     merged_df = pd.concat([merged_df, tmp3])

# print('3번째 통과입니다.')

#### 16. 강북빵상회 : 수집은 되지만 리스트 개수가 0개다.

In [ ]:
import pandas as pd # 판다스
import time # 시간차로 인한 에러방지
import random
from tqdm import tqdm # 코드가 멋져짐
from selenium.webdriver.common.by import By # 경로추적
from selenium.webdriver.common.keys import Keys # 엔터키
from selenium import webdriver # 셀레니움
from numpy import NaN, NAN, nan # 결측치

# 기본 정보 찾기 함수
def find_text(col, url):
    try:
        element = driver.find_element(By.CSS_SELECTOR, url).text
    except:
        element = '정보없음'
    col.append(element)

# 빈데이터프레임, 클래스
info_df = pd.DataFrame()
review_df = pd.DataFrame()
merged_df = pd.DataFrame()
driver = webdriver.Edge()

# 구글 맵 들어가기
url = f'https://www.google.co.kr/maps/?hl=ko&entry=ttu'
driver.get(url)

search_box = driver.find_element(By.XPATH, '//*[@id="searchboxinput"]')
search_box.send_keys("서울특별시 강북빵상")
search_box.send_keys(Keys.ENTER)

time.sleep(random.uniform(3,4))
driver.find_element(By.XPATH, f'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{1}]/div/a').click()

# [step2] 리뷰 버튼 클릭
time.sleep(random.uniform(3,4))
driver.find_element(By.CSS_SELECTOR, 'button:nth-child(2)').click()

reviews = driver.find_elements(By.CSS_SELECTOR, '.wiI7pd')  # 현재 로드된 모든 리뷰를 찾는다. elements로 가져와서 len=1의 리스트가 됐다.

In [ ]:
[reviews.text for i in reviews]

#### 15. [성공] 통합(merge) : 1번 인포 클릭 > 수집 > 디피1 > 리뷰 클릭 > 디피2 > 머지 > 2번 인포 클릭 > 반복

In [1]:
import pandas as pd # 판다스
import time # 시간차로 인한 에러방지
import random
from tqdm import tqdm # 코드가 멋져짐
from selenium.webdriver.common.by import By # 경로추적
from selenium.webdriver.common.keys import Keys # 엔터키
from selenium import webdriver # 셀레니움
from numpy import NaN, NAN, nan # 결측치
from selenium.webdriver.edge.service import Service # 접근제한 우회
from selenium.webdriver.edge.options import Options # 접근제한 우회
from selenium.webdriver.support.ui import WebDriverWait # 기다리기
from selenium.webdriver.support import expected_conditions as EC # 요소 뜰 때까지 대기
from selenium.common.exceptions import TimeoutException # 시간 예외처리

# 기본 정보 찾기 함수
def find_text(col, url):
    try:
        element = driver.find_element(By.CSS_SELECTOR, url).text
    except:
        element = '정보없음'
    col.append(element)

# 빈데이터프레임, 클래스
info_df = pd.DataFrame()
review_df = pd.DataFrame()
merged_df = pd.DataFrame()

# Edge WebDriver의 경로를 지정합니다. '경로/edgedriver'를 실제 경로로 교체해주세요.
service = Service(executable_path='C:\\selenium\\msedgedriver.exe')
options = webdriver.EdgeOptions()
options.use_chromium = True  # Chromium 기반 Edge 사용을 명시합니다.
# 사용자 에이전트 문자열을 설정합니다.
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36"
options.add_argument(f'user-agent={user_agent}')

# Edge 드라이버를 초기화합니다.
driver = webdriver.Edge(service=service, options=options)

# 구글 맵 들어가기
url = f'https://www.google.co.kr/maps/?hl=ko&entry=ttu'
driver.get(url)

time.sleep(random.uniform(2,3))
search_box = driver.find_element(By.XPATH, '//*[@id="searchboxinput"]')
search_box.send_keys("서울특별시 종로구 빵")
search_box.send_keys(Keys.ENTER)

i = 3  # [step1] 인포, 리뷰, "클릭" 순으로 변경한다.
while True:
    time.sleep(random.uniform(3,4))  # 구글 들어가서 대기
    driver.find_element(By.XPATH, f'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{i}]/div/a').click()

    # 주의하자. 반복되는 건 전역변수/지역변수의 문제다.
    name = []
    rating = []
    review = []
    category = []
    adress = []

    time.sleep(random.uniform(2,3))
    find_text(name, ".DUwDvf.lfPIob") # 이름에서 막히면 restart하세요.
    find_text(rating, ".F7nice > span:nth-child(1)")
    find_text(review, ".F7nice > span:nth-child(2)")
    find_text(category, ".fontBodyMedium span button.DkEaL")
    find_text(adress, ".rogA2c")

    # 임시 데이터프레임 (concat된 후 리셋됨)
    time.sleep(random.uniform(2,3))
    tmp = pd.DataFrame()
    tmp['name'] = name
    tmp['rating'] = rating
    tmp['review'] = review
    tmp['category'] = category
    tmp['adress'] = adress

    # 최종 데이터프레임은 전역변수에 들어감 > 빈 DF에 덮어쓰기
    time.sleep(random.uniform(2,3))
    info_df = pd.concat([info_df, tmp])
    
    # [step2] 리뷰 버튼 클릭 전 대기 [청수당, 아티장 에러주의]
    # time.sleep(random.uniform(3,5))
    # driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[3]/div/div/button[2]').click()

    try:
        # WebDriverWait과 expected_conditions를 사용하여 요소가 나타날 때까지 최대 10초간 대기
        elements = WebDriverWait(driver, 60).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[3]/div/div/button[2]')))
        for element in elements:
            element.click()  # 요소가 발견되면 클릭
    # except TimeoutException:
    #     print("해당 요소를 찾는 데 시간이 너무 오래 걸립니다.")
    except:
        time.sleep(random.uniform(3,5))
        driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[3]/div/div/button[2]').click()

    # element = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[3]/div/div/button[2]')
    # driver.execute_script("arguments[0].click();", element)

    # 정렬 버튼 클릭하기 전 대기
    try:
        time.sleep(random.uniform(3,4))
        driver.find_element(By.CSS_SELECTOR, 'div.m6QErb.Pf6ghf.KoSBEe.ecceSd.tLjsW > div.TrU0dc.kdfrQc').click()

        # 최신순 클릭하기 전 대기
        time.sleep(random.uniform(1,3))
        driver.find_element(By.XPATH, '//*[@id="action-menu"]/div[2]').click()

        result = []  # 새로운 리뷰만 넣을 건데, 중복을 제거하는 조건을 넣어야 한다.
        answer = []

        time.sleep(random.uniform(2,4))  # 클릭하고 대기시간 필요하다. [에러주의]
        scroll = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[3]')
        last_height = driver.execute_script("return arguments[0].scrollHeight", scroll)
        while True:
            driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", scroll)
            time.sleep(random.uniform(3, 4))  # 새 리뷰가 로드되기를 기다린다.
            new_height = driver.execute_script("return arguments[0].scrollHeight", scroll)
            if new_height == last_height:
                break
            last_height = new_height

        last_review = None  # 비교하는 변수를 추가한다. WOW
        flag = False  # 종료 플래그를 사용해본다.
        while not flag:
            time.sleep(random.uniform(2, 3))  # 페이지 로드를 기다린다.
            element = driver.find_element(By.CSS_SELECTOR, '#QA0Szd')  # 소름 돋는 점. 필요 없는 코드다.
            reviews = driver.find_elements(By.CSS_SELECTOR, '.wiI7pd')  # 현재 로드된 모든 리뷰를 찾는다. elements로 가져와서 len=1의 리스트가 됐다.
            
            if not reviews:  # 강북상회(1) 리뷰가 없다면 루프를 종료한다.
                break

            for review in reviews:
                review_text = review.text
                if review_text == last_review:  # 마지막으로 확인한 리뷰와 같다면 새 리뷰 로드를 기다린다.
                    break
                if review_text not in result:
                    if review.tag_name.lower() == 'span':
                        result.append(review_text)
                    else:
                        answer.append(review_text)
                else:
                    flag=True
            
            if len(result) == 1:  # 로드된 리뷰가 하나만 있다면, 더 이상 새 리뷰가 없는 것으로 간주하고 루프를 종료한다.
                break
                
            last_review = result[-1] if result else None  # 마지막 리뷰를 확인용 변수에 업데이트 한다. result가 빈값으면(False) None을 쓴다.
        
        # 강북빵상회(2)
        if result:
            # 리뷰 데이터프레임을 만든다.
            tmp2 = pd.DataFrame()
            tmp2['name'] = name*len(result)
            tmp2['review'] = result

            # 리뷰 데이터프레임을 적재한다.
            time.sleep(random.uniform(1,2))
            review_df = pd.concat([review_df, tmp2])

        #     # 머지 데이터프레임을 만들어서 적재한다.
        #     # time.sleep(random.uniform(1,2))
        #     # tmp3 = pd.merge(tmp, tmp2, on="name", how="right")
        #     # merged_df = pd.concat([merged_df, tmp3])
    except:
        # result = []  # 리뷰0개 프린트용으로 추가
        pass  # 정렬 버튼이 없으면 지나가시오.

    # [step3] 다음 가게 번호 부여하고 
    time.sleep(random.uniform(1,2))
    try:
        i+=2
        element = driver.find_element(By.XPATH, f'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{i}]/div/a')
        driver.execute_script("arguments[0].scrollIntoView(true);", element) # element 위치로 이동 > 수집 반복
    except:
        break
    
    # if review == ['정보없음']:
    #     result = []
    print(f"{name}상점에서 {len(result)}개의 리뷰를 수집했습니다.")  # 파머스반브래드 12개> 28개

['한나식빵 응암점']상점에서 37개의 리뷰를 수집했습니다.
['폴스타 베이커스(pole star bakers)']상점에서 69개의 리뷰를 수집했습니다.
['화선 베이커리']상점에서 6개의 리뷰를 수집했습니다.
['빵이야기']상점에서 1개의 리뷰를 수집했습니다.
['쿠아레비 구산점']상점에서 12개의 리뷰를 수집했습니다.
['쿠아레비 역촌점']상점에서 8개의 리뷰를 수집했습니다.
['도돌이표']상점에서 11개의 리뷰를 수집했습니다.
['뚜레쥬르 녹번역점']상점에서 11개의 리뷰를 수집했습니다.
['뚜레쥬르 카페응암점']상점에서 8개의 리뷰를 수집했습니다.
['파리바게뜨 은평행복']상점에서 8개의 리뷰를 수집했습니다.
['몽소 베이커리 & 카페']상점에서 3개의 리뷰를 수집했습니다.
['뚜레쥬르 응암오거리']상점에서 2개의 리뷰를 수집했습니다.
['블랑제리우트']상점에서 4개의 리뷰를 수집했습니다.
['노블티']상점에서 8개의 리뷰를 수집했습니다.
['파리바게뜨']상점에서 29개의 리뷰를 수집했습니다.
['시월구일 베이커리']상점에서 11개의 리뷰를 수집했습니다.
['뚜레쥬르 역촌2동점']상점에서 8개의 리뷰를 수집했습니다.
['이리와바게트']상점에서 1개의 리뷰를 수집했습니다.
['프랑세즈 베이커리']상점에서 1개의 리뷰를 수집했습니다.
['라이언베이커 LION BAKER']상점에서 21개의 리뷰를 수집했습니다.
['데일리베이크하우스']상점에서 3개의 리뷰를 수집했습니다.
['스위츠마인']상점에서 12개의 리뷰를 수집했습니다.
['뚜레쥬르 은평구청점']상점에서 4개의 리뷰를 수집했습니다.
['박성원베이커리']상점에서 5개의 리뷰를 수집했습니다.
['크렘마마']상점에서 6개의 리뷰를 수집했습니다.
['프랑세즈 베이커리 불광점']상점에서 4개의 리뷰를 수집했습니다.
['오늘의 빵']상점에서 1개의 리뷰를 수집했습니다.
['빵드밀역촌점']상점에서 1개의 리뷰를 수집했습니다.
['파리바게뜨']상점에서 29개의 리뷰를 수집했습니다.
['하얀풍차']상점에서 29개의 리뷰

In [ ]:
merged = pd.merge(info_df, review_df, on="name", how="right")
merged

,name,rating,review_x,category,adress,review_y
0,파머스반브레드,5.0,(14),제과점,서울특별시 종로구 돈화문로11가길 59 현대뜨레비앙 111호,정말 맛있는 유럽빵. 아몬드 크로아상 정말 맛있어요! 아몬드 페이스트가 가득 들어있...
1,파머스반브레드,5.0,(14),제과점,서울특별시 종로구 돈화문로11가길 59 현대뜨레비앙 111호,"독일보다 나은 서울의 비트 베이커리, 버터가 들어간 브레첼은 꼭 먹어보세요, 꼭 먹..."
2,파머스반브레드,5.0,(14),제과점,서울특별시 종로구 돈화문로11가길 59 현대뜨레비앙 111호,이 곳 빵은 좋은 제품들로 만들어져서 비싸지만 정말 그 값을 합니다! 많이 먹어도 ...
3,파머스반브레드,5.0,(14),제과점,서울특별시 종로구 돈화문로11가길 59 현대뜨레비앙 111호,"훌륭한 음료 선택, 차분한 분위기. 케이크 선택이 하이라이트입니다. 인테리어도 너무..."
4,파머스반브레드,5.0,(14),제과점,서울특별시 종로구 돈화문로11가길 59 현대뜨레비앙 111호,구수하고 속이 편안했던 시금치 치아바타 👍…
...,...,...,...,...,...,...
7637,빠다롤뺑프랑스,4.3,(379),프랑스 제과점,대구광역시 중구 봉산동 35-96,비싸지만 맛있오! 🤗🤗…
7638,빠다롤뺑프랑스,4.3,(379),프랑스 제과점,대구광역시 중구 봉산동 35-96,공간이 협소한 편이고 식탁도 있지만 자리가 10개 남짓인데 빵이 정말 맛있습니다.
7639,빠다롤뺑프랑스,4.3,(379),프랑스 제과점,대구광역시 중구 봉산동 35-96,맛남. 가격은 조금 비싸지만 또 먹고싶은 빵이 있는곳.
7640,빠다롤뺑프랑스,4.3,(379),프랑스 제과점,대구광역시 중구 봉산동 35-96,2016.11. 친절하고 싼 가격...그러나....


In [ ]:
result = ['정보없음']

if result == ['정보없음']:
    print('맞습니다.')

맞습니다.


In [ ]:
import pandas as pd
result = ['너무 맛있어요', '분위기가 좋아요', '인테리어가 멋있어요', '사람들이 많아 찾아요.']
tmp2 = pd.DataFrame()
tmp2['REVIEW'] = result
tmp2['NAME'] = '솔트24'  # list를 벗겨야 한다.
tmp2

,REVIEW,NAME
0,너무 맛있어요,솔트24
1,분위기가 좋아요,솔트24
2,인테리어가 멋있어요,솔트24
3,사람들이 많아 찾아요.,솔트24


In [ ]:
name = '솔트24'
[name for i in range(5)]

['솔트24', '솔트24', '솔트24', '솔트24', '솔트24']

In [ ]:
[name]*5

['솔트24', '솔트24', '솔트24', '솔트24', '솔트24']

In [11]:
info_df.to_csv('google_maps_info_Eunpyeong.csv', index=False)

In [9]:
info_df

,name,rating,review,category,adress
0,한나식빵 응암점,3.7,(62),제과점,서울특별시 은평구 응암동 가좌로7길 9
0,폴스타 베이커스(pole star bakers),4.3,(124),커피숍/커피 전문점,서울특별시 은평구 서오릉로 256
0,화선 베이커리,3.0,(8),제과점,서울특별시 은평구 연서로26길 6
0,빵이야기,4.0,(1),제과점,서울특별시 은평구 대조동 통일로77길 17
0,쿠아레비 구산점,4.7,(14),제과점,서울특별시 은평구 구산동 서오릉로 149
...,...,...,...,...,...
0,프랭크버거 서울 녹번역점,4.8,(5),햄버거 전문점,서울특별시 은평구 은평로 240
0,산보클럽,5.0,(1),카페,서울특별시 은평구 가좌로11가길 2
0,페스카,4.2,(23),이탈리아 음식점,서울특별시 은평구 대조동 1층 연서로 204
0,이디야커피 백련산아이파크점,4.0,(108),커피숍/커피 전문점,서울특별시 은평구 응암로 264


In [10]:
review_df.to_csv('google_maps_review_Eunpyeong.csv', index=False)

In [3]:
review_df

,name,review
0,한나식빵 응암점,예전같지않네요...
1,한나식빵 응암점,힐링 빵집
2,한나식빵 응암점,유명 매장 빵보다 맛있고 좋은 밀가루 쓰며 값이 싸서 전 굳이 한나식빵을 찾습니다....
3,한나식빵 응암점,너무 맛있어요
4,한나식빵 응암점,우리동네 버스정류장
...,...,...
3,카페포조,대형프랜차이즈 커피보다 더 맛있어요. 조용하게 혼자 커피마시기 좋습니다. 나만 알고...
4,카페포조,안쪽도 깔끔하고 좋았음
5,카페포조,딸기라떼 마셨는데 요근래 먹은 딸기라떼중에 젤 맛있었어요. 많이 달지않고 청도 많이...
6,카페포조,맛있는 수제 빵과 맛있는 커피가 있는 곳.~^^


In [ ]:
# 강북빵상회 스크롤 되고 리뷰가 만들어져도 속이 비었다.
import pandas as pd
tmp2 = pd.DataFrame()
tmp2['review'] = []
tmp2

In [ ]:
# 이상하게도(?) 모든 인덱스 '0'이라서 리셋하기
info_df.reset_index(drop=True, inplace=True)
info_df.tail()

In [ ]:
# 리뷰 데이터프레임을 만든다.
tmp2 = pd.DataFrame({
    "name": name*len(result),
    "review": result
})

In [ ]:
import pandas as pd

# 예시 데이터 생성
# table1과 table2의 구조를 따라서 예시 데이터를 생성합니다.
table1 = pd.DataFrame({
    "store_name": ["가게A", "가게B", "가게C"],
    "phone_number": ["010-1234-5678", "010-2345-6789", "010-3456-7890"],
    "id": [1, 2, 3],
    "adress": ["주소1", "주소2", "주소3"]
})

table2 = pd.DataFrame({
    "review": ["리뷰1", "리뷰2", "리뷰4"],
    "store_name": ["가게A", "가게A", "가게A"]
})

# 두 테이블을 'store_name' 열을 기준으로 병합합니다.
# how 매개변수를 'inner'로 설정하면, 두 테이블 모두에 존재하는 'store_name' 값만을 기준으로 병합합니다.
# 'outer', 'left', 'right' 등 다른 옵션을 사용할 수도 있습니다.
merged_table = pd.merge(table1, table2, on="store_name", how="outer")

print(merged_table)

In [ ]:
table1

In [ ]:
table2

In [ ]:
a = ["아"]
a*3

#### 14. 가드니아가 1부터 시작하네? [성공] 일단 사장님 리뷰를 제거한다. 'span'

In [ ]:
import pandas as pd  # 판다스
import time  # 시간차로 인한 에러방지
import random
from tqdm import tqdm  # 코드가 멋져짐
from selenium.webdriver.common.by import By  # 경로추적
from selenium.webdriver.common.keys import Keys  # 엔터키
from selenium import webdriver  # 셀레니움
from numpy import NaN, NAN, nan  # 결측치

# 기본 정보 찾기 함수
def find_text(col, url):
    try:
        element = driver.find_element(By.CSS_SELECTOR, url).text
    except:
        element = '정보없음'
    col.append(element)

# 빈데이터프레임, 클래스
info_df = pd.DataFrame()
driver = webdriver.Edge()

# 구글 맵 들어가기
url = f'https://www.google.co.kr/maps/?hl=ko&entry=ttu'
driver.get(url)

search_box = driver.find_element(By.XPATH, '//*[@id="searchboxinput"]')
search_box.send_keys("종로구 도토리")
search_box.send_keys(Keys.ENTER)

# 1번째_빵집 클릭
time.sleep(random.uniform(3,5))
driver.find_element(By.XPATH, f'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{3}]/div/a').click()

# 리뷰 버튼 클릭
time.sleep(random.uniform(3,5))
driver.find_element(By.CSS_SELECTOR, 'button:nth-child(2)').click()

# 정렬 버튼 클릭하기
try:
    time.sleep(random.uniform(1,3))
    driver.find_element(By.CSS_SELECTOR, 'div.m6QErb.Pf6ghf.KoSBEe.ecceSd.tLjsW > div.TrU0dc.kdfrQc').click()

    # 최신순 클릭하기
    time.sleep(random.uniform(1,3))
    driver.find_element(By.XPATH, '//*[@id="action-menu"]/div[2]').click()

    result = []  # 새로운 리뷰만 넣을 건데, 중복을 제거하는 조건을 넣어야 한다.
    answer = []

    time.sleep(random.uniform(2,3))  # 클릭하고 대기시간 필요하다. [에러주의]
    scroll = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[3]')
    last_height = driver.execute_script("return arguments[0].scrollHeight", scroll)
    while True:
        driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", scroll)
        time.sleep(random.uniform(3, 4))  # 새 리뷰가 로드되기를 기다린다.
        new_height = driver.execute_script("return arguments[0].scrollHeight", scroll)
        if new_height == last_height:
            break
        last_height = new_height

    last_review = None  # 비교하는 변수를 추가한다. WOW
    flag = False  # 종료 플래그를 사용해본다.
    while not flag:
        time.sleep(random.uniform(2, 3))  # 페이지 로드를 기다린다.
        element = driver.find_element(By.CSS_SELECTOR, '#QA0Szd')  # 소름 돋는 점. 필요 없는 코드다.
        reviews = driver.find_elements(By.CSS_SELECTOR, '.wiI7pd')  # 현재 로드된 모든 리뷰를 찾는다. elements로 가져와서 len=1의 리스트가 됐다.
        
        for review in reviews:
            review_text = review.text
            if review_text == last_review:  # 마지막으로 확인한 리뷰와 같다면 새 리뷰 로드를 기다린다.
                break
            if review_text not in result:
                if review.tag_name.lower() == 'span':
                    result.append(review_text)
                else:
                    answer.append(review_text)
            else:
                flag=True
            
        last_review = result[-1] if result else None  # 마지막 리뷰를 확인용 변수에 업데이트 한다. result가 빈값으면(False) None을 쓴다.
except:
    print("리뷰가 없습니다.") # 차후 pass등으로 교체한다.

print(len(result), "개의 리뷰를 수집했습니다.")

In [ ]:
answer[:3]

In [ ]:
result[:3] # 195개이다. element/driver 개수가 모두 같다.

In [ ]:
# 스크롤 가능한 요소 찾기
scrollable_element = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]')
# 요소의 "class" 속성 값 가져오기
class_value = scrollable_element.get_attribute("class")
# 업체 대표 응답 ID 코드, 태그 네임 (리스트에 그냥 넣어둠)
['QA0Szd', 'span']

#### 13. [성공] 스크롤부터 쫙- 내려볼까? 아이디어 성공이다. (새 리뷰 비교해서 넣기+중복제거, 종료 플래그, 요소끝까지 내리기)

In [ ]:
import pandas as pd  # 판다스
import time  # 시간차로 인한 에러방지
import random
from tqdm import tqdm  # 코드가 멋져짐
from selenium.webdriver.common.by import By  # 경로추적
from selenium.webdriver.common.keys import Keys  # 엔터키
from selenium import webdriver  # 셀레니움
from numpy import NaN, NAN, nan  # 결측치

# 기본 정보 찾기 함수
def find_text(col, url):
    try:
        element = driver.find_element(By.CSS_SELECTOR, url).text
    except:
        element = '정보없음'
    col.append(element)

# 빈데이터프레임, 클래스
info_df = pd.DataFrame()
driver = webdriver.Edge()

# 구글 맵 들어가기
url = f'https://www.google.co.kr/maps/?hl=ko&entry=ttu'
driver.get(url)

search_box = driver.find_element(By.XPATH, '//*[@id="searchboxinput"]')
search_box.send_keys("서울특별시 종로구 빵")
search_box.send_keys(Keys.ENTER)

# 1번째_빵집 클릭
time.sleep(random.uniform(3,5))
driver.find_element(By.XPATH, f'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{3}]/div/a').click()

# 리뷰 버튼 클릭
time.sleep(random.uniform(3,5))
driver.find_element(By.CSS_SELECTOR, 'button:nth-child(2)').click()

# 정렬 버튼 클릭하기
try:
    time.sleep(random.uniform(1,3))
    driver.find_element(By.CSS_SELECTOR, 'div.m6QErb.Pf6ghf.KoSBEe.ecceSd.tLjsW > div.TrU0dc.kdfrQc').click()

    # 최신순 클릭하기
    time.sleep(random.uniform(1,3))
    driver.find_element(By.XPATH, '//*[@id="action-menu"]/div[2]').click()

    result = []  # 새로운 리뷰만 넣을 건데, 중복을 제거하는 조건을 넣어야 한다.

    time.sleep(random.uniform(2,3))  # 클릭하고 대기시간 필요하다. [에러주의]
    scroll = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[3]')
    last_height = driver.execute_script("return arguments[0].scrollHeight", scroll)
    while True:
        driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", scroll)
        time.sleep(random.uniform(3, 4))  # 새 리뷰가 로드되기를 기다린다.
        new_height = driver.execute_script("return arguments[0].scrollHeight", scroll)
        if new_height == last_height:
            break
        last_height = new_height

    last_review = None  # 비교하는 변수를 추가한다. WOW
    flag = False  # 종료 플래그를 사용해본다.
    while not flag:
        time.sleep(random.uniform(2, 3))  # 페이지 로드를 기다린다.
        element = driver.find_element(By.CSS_SELECTOR, '#QA0Szd')  # 소름 돋는 점. 필요 없는 코드다.
        reviews = driver.find_elements(By.CSS_SELECTOR, '.wiI7pd')  # 현재 로드된 모든 리뷰를 찾는다. elements로 가져와서 len=1의 리스트가 됐다.
        
        for review in reviews:
            review_text = review.text
            if review_text == last_review:  # 마지막으로 확인한 리뷰와 같다면 새 리뷰 로드를 기다린다.
                break
            if review_text not in result:
                result.append(review_text)
            else:
                flag=True
            
        last_review = result[-1] if result else None  # 마지막 리뷰를 확인용 변수에 업데이트 한다. result가 빈값으면(False) None을 쓴다.
except:
    print("리뷰가 없습니다.") # 차후 pass등으로 교체한다.

print(len(result), "개의 리뷰를 수집했습니다.")

In [ ]:
for i, review in enumerate(result):
    print(i) # 개행필요?
    print(review)

#### 13-2. 참고코드 : 페이지를 끝까지 내리기, 정희는 요소 비교하며 스크롤하기, 나는 정희+새로운 코드

In [ ]:
# 마지막으로 측정된 스크롤 높이를 저장할 변수입니다.
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # 페이지 끝까지 스크롤합니다.
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # 페이지가 로드되기를 기다립니다.
    time.sleep(3)

    # 새로운 스크롤 높이를 측정하고 이전 높이와 비교합니다.
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        # 높이가 변하지 않았다면, 스크롤이 끝에 도달했음을 의미하므로 반복을 종료합니다.
        break
    last_height = new_height

#### 13-1. 참고코드: 효율적으로 15개 리뷰만 중복 수집 방지하기

In [ ]:
result = []
last_len = 0  # 마지막에 확인한 result 리스트의 길이

while True:
    time.sleep(random.uniform(2, 3))  # 페이지 로드를 기다립니다.
    
    reviews = driver.find_elements(By.CSS_SELECTOR, '.wiI7pd')  # 현재 로드된 모든 리뷰를 찾습니다.
    
    for review in reviews[-15:]:  # 마지막 15개 리뷰만 확인하여 중복 수집 방지
        review_text = review.text
        if review_text not in result:  # 중복되지 않는 리뷰만 result에 추가
            result.append(review_text)
    
    if len(result) == last_len:  # 새로운 리뷰가 추가되지 않았다면 반복 종료
        print("새로운 리뷰가 더 이상 없습니다.")
        break
    else:
        last_len = len(result)  # result 리스트의 길이 업데이트

#### 12. 리뷰야 너 예전꺼랑 같은거니? 새거 수집해

In [ ]:
import pandas as pd # 판다스
import time # 시간차로 인한 에러방지
import random
from tqdm import tqdm # 코드가 멋져짐
from selenium.webdriver.common.by import By # 경로추적
from selenium.webdriver.common.keys import Keys # 엔터키
from selenium import webdriver # 셀레니움
from numpy import NaN, NAN, nan # 결측치

# 기본 정보 찾기 함수
def find_text(col, url):
    try:
        element = driver.find_element(By.CSS_SELECTOR, url).text
    except:
        element = '정보없음'
    col.append(element)

# 빈데이터프레임, 클래스
info_df = pd.DataFrame()
driver = webdriver.Edge()

# 구글 맵 들어가기
url = f'https://www.google.co.kr/maps/?hl=ko&entry=ttu'
driver.get(url)

search_box = driver.find_element(By.XPATH, '//*[@id="searchboxinput"]')
search_box.send_keys("서울특별시 종로구 빵")
search_box.send_keys(Keys.ENTER)

# 1번째_빵집 클릭
time.sleep(random.uniform(3,5))
driver.find_element(By.XPATH, f'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{5}]/div/a').click()

# 리뷰 버튼 클릭
time.sleep(random.uniform(2,3))
driver.find_element(By.CSS_SELECTOR, 'button:nth-child(2)').click()

result = [] # 새로운 리뷰만 넣는데 중복을 제거하는 조건을 넣어야 한다.
last_review = None # 비교하는 변수를 추가한다. WOW
flag = False

while not flag:
    time.sleep(random.uniform(2, 3))  # 페이지 로드를 기다린다.
    element = driver.find_element(By.CSS_SELECTOR, '#QA0Szd')
    reviews = driver.find_elements(By.CSS_SELECTOR, '.wiI7pd')  # 현재 로드된 모든 리뷰를 찾는다. elements로 가져와서 len=1의 리스트가 됐다.
    
    for review in reviews:
        review_text = review.text
        if review_text == last_review:  # 마지막으로 확인한 리뷰와 같다면 새 리뷰 로드를 기다린다.
            break
        if review_text not in result:
            result.append(review_text)
        else:
            flag=True
        
    last_review = result[-1] if result else None  # 마지막 리뷰를 확인용 변수에 업데이트 한다.
    
    try:
        scroll = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[3]')
        driver.execute_script("arguments[0].scrollTop += 700;", scroll)
        time.sleep(random.uniform(4, 5))  # 새 리뷰가 로드되기를 기다린다.
    except Exception as e:
        print("스크롤 중 에러 발생:", e)
        break # 여기서 멈추지 않는 문제가 발생했다!

print(len(result), "개의 리뷰를 수집했습니다.")

In [ ]:
a = [1,2,3,4,5,6,7]
a[-15:] # 범위 오류는 안나는 구나

#### 11. 리뷰 수집하기의 문제: 하나의 리뷰만 반복해서 수집한다. (로드 시간이 필요하다?)

In [ ]:
import pandas as pd # 판다스
import time # 시간차로 인한 에러방지
import random
from tqdm import tqdm # 코드가 멋져짐
from selenium.webdriver.common.by import By # 경로추적
from selenium.webdriver.common.keys import Keys # 엔터키
from selenium import webdriver # 셀레니움
from numpy import NaN, NAN, nan # 결측치

# 기본 정보 찾기 함수
def find_text(col, url):
    try:
        element = driver.find_element(By.CSS_SELECTOR, url).text
    except:
        element = '정보없음'
    col.append(element)

# 빈데이터프레임, 클래스
info_df = pd.DataFrame()
driver = webdriver.Edge()

# 구글 맵 들어가기
url = f'https://www.google.co.kr/maps/?hl=ko&entry=ttu'
driver.get(url)

search_box = driver.find_element(By.XPATH, '//*[@id="searchboxinput"]')
search_box.send_keys("서울특별시 종로구 빵")
search_box.send_keys(Keys.ENTER)

# 1번째_빵집 클릭
time.sleep(random.uniform(3,5))
driver.find_element(By.XPATH, f'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{3}]/div/a').click()

# 리뷰 버튼 클릭
time.sleep(random.uniform(2,3))
driver.find_element(By.CSS_SELECTOR, 'button:nth-child(2)').click()

# .wiI7pd span
# QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf > div:nth-child(8) > div:nth-child(1) > div > div > div:nth-child(4) > div:nth-child(2)

# 리뷰 수집하기
result = []
while True:
    element = driver.find_element(By.CSS_SELECTOR, '#QA0Szd')
    time.sleep(random.uniform(2,3))
    review = element.find_element(By.CSS_SELECTOR, '.wiI7pd').text
    result.append(review)
    try:
        time.sleep(random.uniform(2,3))
        element = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[3]')
        driver.execute_script("arguments[0].scrollTop += 3000;", element) # element 위치로 이동 반복
    except:
        break

In [ ]:
for i in element2:
    review = i.find_element(By.CSS_SELECTOR, '.wiI7pd').text
    print(review)

1. 셀레니움 단일객체는 반복되지 않는다.
2. text로 변환하면 1개가 된다.
3. elements로 가져오면 리스트인데 len=1이다.

In [ ]:
element.text

In [ ]:
review = element.find_element(By.CSS_SELECTOR, '.wiI7pd').text

In [ ]:
print(review)

In [ ]:
len(element.text) # 한글자씩 쪼개서 센다.

#### 10. 리뷰 스크롤 수집하고 내리기 (내리기만 성공한 상태)

In [ ]:
import pandas as pd # 판다스
import time # 시간차로 인한 에러방지
import random
from tqdm import tqdm # 코드가 멋져짐
from selenium.webdriver.common.by import By # 경로추적
from selenium.webdriver.common.keys import Keys # 엔터키
from selenium import webdriver # 셀레니움
from numpy import NaN, NAN, nan # 결측치

# 기본 정보 찾기 함수
def find_text(col, url):
    try:
        element = driver.find_element(By.CSS_SELECTOR, url).text
    except:
        element = '정보없음'
    col.append(element)

# 빈데이터프레임, 클래스
info_df = pd.DataFrame()
driver = webdriver.Edge()

# 구글 맵 들어가기
url = f'https://www.google.co.kr/maps/?hl=ko&entry=ttu'
driver.get(url)

search_box = driver.find_element(By.XPATH, '//*[@id="searchboxinput"]')
search_box.send_keys("서울특별시 종로구 빵")
search_box.send_keys(Keys.ENTER)

# 1번째_빵집 클릭
time.sleep(random.uniform(3,5))
driver.find_element(By.XPATH, f'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{3}]/div/a').click()

# 리뷰 버튼 클릭
time.sleep(random.uniform(2,3))
driver.find_element(By.CSS_SELECTOR, 'button:nth-child(2)').click()

# driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
# driver.execute_script("arguments[0].scrollTop += 600;", scrollable_element)
# driver.execute_script("return arguments[0].scrollHeight", scrollable_element)
# driver.execute_script("arguments[0].scrollIntoView(true);", element)
# (By.XPATH,'//*[@id="QA0Szd"]/div/div/div[1]/div[3]')

# 수집은 CSS지만 이동은 잘하니까 믿어본다. > ifram이 필요없으니까 없는거야 > 없는 건 그만 찾자.
while True:
    # temp = []
    # find_text(temp, ".DUwDvf.lfPIob")
    
    try:
        time.sleep(random.uniform(2,3))
        element = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[3]')
        driver.execute_script("arguments[0].scrollTop += 500;", element) # element 위치로 이동 반복
    except:
        break

#### 9. [성공] 기본정보 찾기 > 검색어 입력까지 추가 (우회없음)

In [ ]:
import pandas as pd # 판다스
import time # 시간차로 인한 에러방지
import random
from tqdm import tqdm # 코드가 멋져짐
from selenium.webdriver.common.by import By # 경로추적
from selenium.webdriver.common.keys import Keys # 엔터키
from selenium import webdriver # 셀레니움
from numpy import NaN, NAN, nan # 결측치

# 기본 정보 찾기 함수
def find_text(col, url):
    try:
        element = driver.find_element(By.CSS_SELECTOR, url).text
    except:
        element = '정보없음'
    col.append(element)

# 빈데이터프레임, 클래스
info_df = pd.DataFrame()
driver = webdriver.Edge()

# 구글 맵 들어가기
url = f'https://www.google.co.kr/maps/?hl=ko&entry=ttu'
driver.get(url)

search_box = driver.find_element(By.XPATH, '//*[@id="searchboxinput"]')
search_box.send_keys("서울특별시 종로구 빵")
search_box.send_keys(Keys.ENTER)

i = 3
while True:
    time.sleep(random.uniform(2,3))
    driver.find_element(By.XPATH, f'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{i}]/div/a').click()

    # 주의하자. 반복되는 건 전역변수/지역변수의 문제다.
    name = []
    rating = []
    review = []
    category = []
    adress = []

    time.sleep(random.uniform(1,3))
    find_text(name, ".DUwDvf.lfPIob") # 이름에서 막히면 restart하세요.
    find_text(rating, ".F7nice > span:nth-child(1)")
    find_text(review, ".F7nice > span:nth-child(2)")
    find_text(category, ".fontBodyMedium span button.DkEaL")
    find_text(adress, ".rogA2c")

    # 임시 데이터프레임 (concat된 후 리셋됨)
    time.sleep(random.uniform(1,3))
    tmp = pd.DataFrame()
    tmp['name'] = name
    tmp['rating'] = rating
    tmp['review'] = review
    tmp['category'] = category
    tmp['adress'] = adress

    # 최종 데이터프레임은 전역변수에 들어감 > 빈 DF에 덮어쓰기
    time.sleep(random.uniform(1,3))
    info_df = pd.concat([info_df, tmp])
    
    try:
        i+=2
        element = driver.find_element(By.XPATH, f'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{i}]/div/a')
        driver.execute_script("arguments[0].scrollIntoView(true);", element) # element 위치로 이동 > 수집 반복
    except:
        break

# 이상하게도(?) 모든 인덱스 '0'이라서 리셋하기
info_df.reset_index(drop=True, inplace=True)
info_df.tail()

#### 8. 클래스 속성 찾기

In [ ]:
import pandas as pd
import time
import random
from tqdm import tqdm
from selenium.webdriver.common.by import By
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.edge.service import Service # 접근제한 우회
from selenium.webdriver.edge.options import Options # 접근제한 우회

def switch_left():
    driver.switch_to.default_content()
    iframe = driver.find_element(By.XPATH,'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]')
    driver.switch_to.frame(iframe)
    
def switch_right():
    driver.switch_to.default_content()
    iframe = driver.find_element(By.XPATH,'//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[3]')
    driver.switch_to.frame(iframe)

# 사용자 에이전트를 변경하기 위한 옵션 설정
options = Options()
options.add_argument("user-agent= Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582")

# Edge 드라이버 서비스
service = Service(executable_path="C:\\selenium\\msedgedriver.exe")

# Edge 드라이버 초기화 시 옵션 적용
driver = webdriver.Edge(service=service, options=options)

# "서울특별시 종로구 빵" 검색 페이지 바로 들어가기(=동적 페이지)
url = f'https://www.google.co.kr/maps/search/%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C+%EC%A2%85%EB%A1%9C%EA%B5%AC+%EB%B9%B5/data=!3m1!4b1?hl=ko&entry=ttu'
driver.get(url)

# n번째_빵집 클릭
time.sleep(random.uniform(1,3))
driver.find_element(By.XPATH, f'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{3}]/div/a').click()

# 리뷰 버튼 클릭
time.sleep(random.uniform(1,3))
driver.find_element(By.CSS_SELECTOR, 'button:nth-child(2)').click()

# 스크롤 가능한 요소 찾기
scrollable_element = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]')

# 요소의 "class" 속성 값 가져오기
class_value = scrollable_element.get_attribute("class")

print(class_value)

#### 7. 스크롤만 내려보기 프레임1/프레임2 (안됨)

In [ ]:
import pandas as pd
import time
import random
from tqdm import tqdm
from selenium.webdriver.common.by import By
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.edge.service import Service # 접근제한 우회
from selenium.webdriver.edge.options import Options # 접근제한 우회

def switch_left():
    driver.switch_to.default_content()
    iframe = driver.find_element(By.XPATH,'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]')
    driver.switch_to.frame(iframe)
    
def switch_right():
    driver.switch_to.default_content()
    iframe = driver.find_element(By.XPATH,'//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[3]')
    driver.switch_to.frame(iframe)

# 사용자 에이전트를 변경하기 위한 옵션 설정
options = Options()
options.add_argument("user-agent= Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582")

# Edge 드라이버 서비스
service = Service(executable_path="C:\\selenium\\msedgedriver.exe")

# Edge 드라이버 초기화 시 옵션 적용
driver = webdriver.Edge(service=service, options=options)

# "서울특별시 종로구 빵" 검색 페이지 바로 들어가기(=동적 페이지)
url = f'https://www.google.co.kr/maps/search/%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C+%EC%A2%85%EB%A1%9C%EA%B5%AC+%EB%B9%B5/data=!3m1!4b1?hl=ko&entry=ttu'
driver.get(url)

# n번째_빵집 클릭
time.sleep(random.uniform(1,3))
driver.find_element(By.XPATH, f'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{3}]/div/a').click()

# 리뷰 버튼 클릭
time.sleep(random.uniform(1,3))
driver.find_element(By.CSS_SELECTOR, 'button:nth-child(2)').click()

# 리뷰를 내려가며 수집하기 (CLASS_NAME 참고)
scrollable_element = driver.find_element(By.XPATH,'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]')
class_name = scrollable_element.get_attribute("class")

last_height = driver.execute_script("return document.body.scrollHeight", scrollable_element)

while True:
    # 페이지 끝까지 스크롤 다운
    # driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # 요소 내에서 아래로 600px 스크롤
    driver.execute_script("arguments[0].scrollTop += 1500;", scrollable_element)
    # 페이지 로드를 기다림
    time.sleep(random.uniform(2,3))
    # 새로운 스크롤 높이 계산 및 비교
    new_height = driver.execute_script("return document.body.scrollHeight", scrollable_element)

    if new_height == last_height:
        break
    last_height = new_height

#### 6. 스크롤 추가하기 (안됨)

In [ ]:
import pandas as pd # 판다스
import time # 시간차로 인한 에러방지
import random # 시간차로 인한 에러방지
from tqdm import tqdm # 코드가 멋져짐
from selenium.webdriver.common.by import By # 경로추적
from selenium import webdriver # 셀레니움
from bs4 import BeautifulSoup # 뷰숲
from selenium.webdriver.edge.service import Service # 에이전드 문자열
from selenium.webdriver.edge.options import Options # 에이전드 문자열

# 사용자 에이전트를 변경하기 위한 옵션 설정
options = Options()
options.add_argument("user-agent= Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582")

# Edge 드라이버 서비스
service = Service(executable_path="C:\\selenium\\msedgedriver.exe")

# Edge 드라이버 초기화 시 옵션 적용
driver = webdriver.Edge(service=service, options=options)

# "서울특별시 종로구 빵" 검색 페이지 바로 들어가기(=동적 페이지)
url = f'https://www.google.co.kr/maps/search/%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C+%EC%A2%85%EB%A1%9C%EA%B5%AC+%EB%B9%B5/data=!3m1!4b1?hl=ko&entry=ttu'
driver.get(url)

# n번째_빵집 클릭
time.sleep(random.uniform(1,3))
driver.find_element(By.XPATH, f'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{3}]/div/a').click()

# 이름 수집
time.sleep(random.uniform(1,3))
element = driver.find_element(By.CSS_SELECTOR, ".DUwDvf.lfPIob")

# 리뷰 버튼 클릭
time.sleep(random.uniform(1,3))
driver.find_element(By.CSS_SELECTOR, 'button:nth-child(2)').click()

last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # 정렬 버튼 클릭하기
    # time.sleep(random.uniform(1,3))
    # driver.find_element(By.CSS_SELECTOR, 'div.m6QErb.Pf6ghf.KoSBEe.ecceSd.tLjsW > div.TrU0dc.kdfrQc').click()

    # 최신순 클릭하기
    # time.sleep(random.uniform(1,3))
    # driver.find_element(By.XPATH, '//*[@id="action-menu"]/div[2]').click()

    # 현재 보고 있는 페이지의 "HTML 소스" 가져오기
    html_cotent = driver.page_source
    # soup 객체는 HTML 문서의 구조에 접근하고 검색할 수 있는 다양한 메소드를 제공함.
    soup = BeautifulSoup(html_cotent, 'html.parser')
    # soup 객체의 find_all 메소드를 사용하여 HTML 문서 내에서 class 속성이 'wiI7pd'인 모든 span 태그를 찾는다.
    # 찾아진 모든 요소는 reviews 리스트에 저장된다. 이 리스트는 후속 처리를 위해 반복문에서 사용된다.
    reviews = soup.find_all('span', class_ ='wiI7pd')

    time.sleep(random.uniform(1,3))
    # 현재 span 태그(review)의 텍스트 내용을 출력한다. .text 속성은 태그 내의 모든 텍스트를 추출한다.
    for i, review in enumerate(reviews):
        print(i) # 개행필요?
        print(review.text)

    # Selenium으로 페이지를 로드하고 BeautifulSoup으로 HTML을 파싱하여 원하는 정보를 찾아내는 과정이다.

    # 페이지 끝까지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # 페이지 로드를 기다림
    time.sleep(random.uniform(2,3))

    # 새로운 스크롤 높이 계산 및 비교
    new_height = driver.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
        break
    last_height = new_height

#### 5. 튜터님의 (뷰숲) 리뷰 코드

In [ ]:
import pandas as pd # 판다스
import time # 시간차로 인한 에러방지
import random # 시간차로 인한 에러방지
import requests # 웹스크래핑
from tqdm import tqdm # 코드가 멋져짐
from selenium.webdriver.common.by import By # 경로추적
from selenium import webdriver # 셀레니움
from bs4 import BeautifulSoup # 뷰숲

driver = webdriver.Edge()

# "서울특별시 종로구 빵" 검색 페이지 바로 들어가기(=동적 페이지)
url = f'https://www.google.co.kr/maps/search/%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C+%EC%A2%85%EB%A1%9C%EA%B5%AC+%EB%B9%B5/data=!3m1!4b1?hl=ko&entry=ttu'
driver.get(url)

# n번째_빵집 클릭
time.sleep(random.uniform(1,3))
driver.find_element(By.XPATH, f'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{3}]/div/a').click()

# 이름 수집
time.sleep(random.uniform(1,3))
element = driver.find_element(By.CSS_SELECTOR, ".DUwDvf.lfPIob")

# 리뷰 버튼 클릭
time.sleep(random.uniform(1,3))
driver.find_element(By.CSS_SELECTOR, 'button:nth-child(2)').click()

# 정렬 버튼 클릭하기
# time.sleep(random.uniform(1,3))
# driver.find_element(By.CSS_SELECTOR, 'div.m6QErb.Pf6ghf.KoSBEe.ecceSd.tLjsW > div.TrU0dc.kdfrQc').click()

# 최신순 클릭하기
# time.sleep(random.uniform(1,3))
# driver.find_element(By.XPATH, '//*[@id="action-menu"]/div[2]').click()

# 현재 보고 있는 페이지의 "HTML 소스" 가져오기
html_cotent = driver.page_source
# soup 객체는 HTML 문서의 구조에 접근하고 검색할 수 있는 다양한 메소드를 제공함.
soup = BeautifulSoup(html_cotent, 'html.parser')
# soup 객체의 find_all 메소드를 사용하여 HTML 문서 내에서 class 속성이 'wiI7pd'인 모든 span 태그를 찾는다.
# 찾아진 모든 요소는 reviews 리스트에 저장된다. 이 리스트는 후속 처리를 위해 반복문에서 사용된다.
reviews = soup.find_all('span', class_ ='wiI7pd')

time.sleep(random.uniform(1,3))
# 현재 span 태그(review)의 텍스트 내용을 출력한다. .text 속성은 태그 내의 모든 텍스트를 추출한다.
for i, review in enumerate(reviews):
    print(i) # 개행필요?
    print(review.text)

# Selenium으로 페이지를 로드하고 BeautifulSoup으로 HTML을 파싱하여 원하는 정보를 찾아내는 과정이다.

#### 4. 빵집 정보만 가져오기 > 하나의 빵집은 크롤링 되는데 연결 시 에러 > 크롤링마다 데이터프레임으로 > 인덱스 문제도 해결

In [ ]:
import pandas as pd # 판다스
import time # 시간차로 인한 에러방지
import random
from tqdm import tqdm # 코드가 멋져짐
from selenium.webdriver.common.by import By # 경로추적
from selenium import webdriver # 셀레니움
from numpy import NaN, NAN, nan # 결측치

def find_text(col, url):
    try:
        element = driver.find_element(By.CSS_SELECTOR, url).text
    except:
        element = '정보없음'
    col.append(element)

info_df = pd.DataFrame()
driver = webdriver.Edge()

url = f'https://www.google.co.kr/maps/search/%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C+%EC%A2%85%EB%A1%9C%EA%B5%AC+%EB%B9%B5/data=!3m1!4b1?hl=ko&entry=ttu'
driver.get(url)

for i in tqdm(range(3,102,2)): # 스크롤 없이 5개만 됨. > 101(rnage102)까지 수집하면 딱 50개다.
    time.sleep(random.uniform(1,3))
    driver.find_element(By.XPATH, f'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{i}]/div/a').click()

    # 주의하자. 반복되는 건 전역변수/지역변수의 문제다.
    name = []
    rating = []
    review = []
    category = []
    adress = []

    time.sleep(random.uniform(1,3))
    find_text(name, ".DUwDvf.lfPIob") # 이름에서 막히면 restart하세요.
    find_text(rating, ".F7nice > span:nth-child(1)")
    find_text(review, ".F7nice > span:nth-child(2)")
    find_text(category, ".fontBodyMedium span button.DkEaL")
    find_text(adress, ".rogA2c")

    time.sleep(random.uniform(1,3))
    # 임시 데이터프레임 (concat된 후 리셋됨)
    tmp = pd.DataFrame()
    tmp['name'] = name
    tmp['rating'] = rating
    tmp['review'] = review
    tmp['category'] = category
    tmp['adress'] = adress

    # 최종 데이터프레임은 전역변수에 들어감.
    time.sleep(random.uniform(1,3))
    info_df = pd.concat([info_df, tmp])  # 빈 데이터프레임에 덮어쓰기

    element = driver.find_element(By.XPATH, f'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{i+2}]/div/a')
    driver.execute_script("arguments[0].scrollIntoView(true);", element) # element 위치로 이동 > 수집 반복

In [ ]:
# info_df.reset_index(drop=True, inplace=True)
info_df.tail()

In [ ]:
[i for i in range(3,8,2)]

#### 3. 정렬 > 최신순 누르기 > 리뷰에서 막힘

In [ ]:
import pandas as pd # 판다스
import time # 시간차로 인한 에러방지
import random
from tqdm import tqdm # 코드가 멋져짐
from selenium.webdriver.common.by import By # 경로추적
from selenium import webdriver # 셀레니움
from numpy import NaN, NAN, nan # 결측치

driver = webdriver.Edge()

url = f'https://www.google.co.kr/maps/search/%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C+%EC%A2%85%EB%A1%9C%EA%B5%AC+%EB%B9%B5/data=!3m1!4b1?hl=ko&entry=ttu'
driver.get(url)

time.sleep(random.uniform(1,3))
driver.find_element(By.XPATH, f'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{3}]/div/a').click()

time.sleep(random.uniform(1,3))
element1 = driver.find_element(By.CSS_SELECTOR, ".DUwDvf.lfPIob") # 이름
element2 = driver.find_element(By.CSS_SELECTOR, ".F7nice > span:nth-child(1)") # 평점
element3 = driver.find_element(By.CSS_SELECTOR, ".F7nice > span:nth-child(2)") # 리뷰수
element4 = driver.find_element(By.CSS_SELECTOR, ".fontBodyMedium span button.DkEaL") # 카테고리
element5 = driver.find_element(By.CSS_SELECTOR, ".rogA2c") # 주소

# 리뷰버튼 (셀릭터의 마지막꺼 쓰면 되네?)
# QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div:nth-child(3) > div > div > button:nth-child(2)
time.sleep(random.uniform(1,3))
driver.find_element(By.CSS_SELECTOR, 'button:nth-child(2)').click()

# 정렬 버튼 클릭하기
# QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf > div.m6QErb.Pf6ghf.KoSBEe.ecceSd.tLjsW > div.TrU0dc.kdfrQc
time.sleep(random.uniform(1,3))
driver.find_element(By.CSS_SELECTOR, 'div.m6QErb.Pf6ghf.KoSBEe.ecceSd.tLjsW > div.TrU0dc.kdfrQc').click()

# 최신순 클릭하기
time.sleep(random.uniform(1,3))
driver.find_element(By.XPATH, '//*[@id="action-menu"]/div[2]').click()

# 리뷰 크롤링 (.GHT2ce.MyEned sapn.wiI7pd')
# QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf > div:nth-child(8)
result = driver.find_element(By.CSS_SELECTOR, f'div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf > div:nth-child(8)')
#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf > div:nth-child(8)

# 다음 가게로(얻어걸림)
# driver.find_element(By.CSS_SELECTOR, '.w6VYqd').click()

# driver.quit() 이거하면 에러난다.
type(result)

In [ ]:
len(result)

In [ ]:
result.text

In [ ]:
result[0].text

In [ ]:
[i.text for i in result]

#### 2. 기본 정보만 가져온 코드

In [ ]:
import pandas as pd # 판다스
import time # 시간차로 인한 에러방지
import random # 시간차로 인한 에러방지
from tqdm import tqdm # 코드가 멋져짐
from selenium.webdriver.common.by import By # 경로추적
from selenium import webdriver # 셀레니움
from bs4 import BeautifulSoup # 뷰숲
from selenium.webdriver.edge.service import Service # 에이전드 문자열
from selenium.webdriver.edge.options import Options # 에이전드 문자열

# 사용자 에이전트를 변경하기 위한 옵션 설정
options = Options()
options.add_argument("user-agent= Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582")

# Edge 드라이버 서비스
service = Service(executable_path="C:\\selenium\\msedgedriver.exe")

# Edge 드라이버 초기화 시 옵션 적용
driver = webdriver.Edge(service=service, options=options)

url = f'https://www.google.co.kr/maps/search/%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C+%EC%A2%85%EB%A1%9C%EA%B5%AC+%EB%B9%B5/data=!3m1!4b1?hl=ko&entry=ttu'
driver.get(url)

time.sleep(random.uniform(1,3))
driver.find_element(By.XPATH, f'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{3}]/div/a').click()

time.sleep(random.uniform(1,3))
element1 = driver.find_element(By.CSS_SELECTOR, ".DUwDvf.lfPIob") # 이름
element2 = driver.find_element(By.CSS_SELECTOR, ".F7nice > span:nth-child(1)") # 평점
element3 = driver.find_element(By.CSS_SELECTOR, ".F7nice > span:nth-child(2)") # 리뷰수
element4 = driver.find_element(By.CSS_SELECTOR, ".fontBodyMedium span button.DkEaL") # 카테고리
element5 = driver.find_element(By.CSS_SELECTOR, ".rogA2c") # 주소

#### 1. xpath만 써 본 코드

In [ ]:
import pandas as pd # 판다스
import time # 시간차로 인한 에러방지
import random # 시간차로 인한 에러방지
from tqdm import tqdm # 코드가 멋져짐
from selenium.webdriver.common.by import By # 경로추적
from selenium import webdriver # 셀레니움
from bs4 import BeautifulSoup # 뷰숲
from selenium.webdriver.edge.service import Service # 에이전드 문자열
from selenium.webdriver.edge.options import Options # 에이전드 문자열

# 사용자 에이전트를 변경하기 위한 옵션 설정
options = Options()
options.add_argument("user-agent= Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582")

# Edge 드라이버 서비스
service = Service(executable_path="C:\\selenium\\msedgedriver.exe")

# Edge 드라이버 초기화 시 옵션 적용
driver = webdriver.Edge(service=service, options=options)

info_df = pd.DataFrame()
review_df = pd.DataFrame()

def find_one_text_list(col, url):
    element = driver.find_element(By.XPATH, url).text
    col.append(element)

# 일단 기본정보를 하나씩 수집하니 하나씩 가져와서 리스트에 넣고 데이터프레임으로 만들까? 되니까.
# elements는 리스트로 가져옴. 근데 다 .text해주는 과정이 필요하다.
# 텍스트를 리스트에 하나씩 쌓고 그걸 임시 데이터프레임의 열로 만들고 cocnat하는 거다.

url = f'https://www.google.co.kr/maps/search/%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C+%EC%A2%85%EB%A1%9C%EA%B5%AC+%EB%B9%B5/data=!3m1!4b1?hl=ko&entry=ttu'
driver.get(url)

time.sleep(random.uniform(1,3))
for i in tqdm(range(3,8,2)):
    time.sleep(1)
    driver.find_element(By.XPATH, f'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{i}]/div/a').click()

    name = []
    rating = []
    review = []
    category = []
    adress = []

    find_one_text_list(name, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div')
    print(name) # 빈 값 나오고 다 오류 뜬다. 접근 제한이 문제가 아니다.
    find_one_text_list(rating, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div/div[1]/div[2]/div/div[1]/div[2]/span[1]/span[1]')
    find_one_text_list(review,'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div/div[1]/div[2]/div/div[1]/div[2]/span[2]/span/span')
    find_one_text_list(category, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div/div[1]/div[2]/div/div[2]/span/span/button')
    find_one_text_list(adress, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[9]/div[3]/button/div/div[2]/div[1]')

    # 임시 데이터프레임 (concat된 후 리셋됨)
    tmp = pd.DataFrame()
    tmp['name'] = name
    tmp['rating'] = rating
    tmp['review'] = review
    tmp['category'] = category
    tmp['adress'] = adress

    # 최종 데이터프레임은 전역변수에 들어감.
    time.sleep(1)
    info_df = pd.concat([info_df, tmp])  # 빈 데이터프레임에 덮어쓰기

    driver.back()

display(info_df)

In [ ]:
[ i for i in range(3,8,2)]

In [ ]:
adress = []
adress.append('아') # 하나의 요소도 리스트에 들어간다.
adress